In [ ]:
!python -V
import sys

import torch

print("Python OK")
try:
    import torch
    import torchvision

    print("Torch:", torch.__version__)
    print("Torchvision:", torchvision.__version__)
except Exception as e:
    print("Torch import error:", e)


!nvidia-smi || echo "NO_GPU"

Python 3.12.12
Python OK
Torch: 2.8.0+cu126
Torchvision: 0.23.0+cu126
Sun Oct 19 09:11:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   64C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |  

In [ ]:

!pip -q install kaggle

from google.colab import files

print("➡️ لطفاً فایل kaggle.json را آپلود کنید…")
uploaded = files.upload()

import json
import os

os.makedirs("/root/.kaggle", exist_ok=True)
if "kaggle.json" not in uploaded:
    raise SystemExit("❌ kaggle.json آپلود نشد. دوباره اجرا و فایل را انتخاب کن.")
with open("/root/.kaggle/kaggle.json", "wb") as f:
    f.write(uploaded["kaggle.json"])
!chmod 600 /root/.kaggle/kaggle.json


!kaggle datasets list -s "cbis-ddsm" | head -n 5 || echo "Kaggle auth failed"

➡️ لطفاً فایل kaggle.json را آپلود کنید…


Saving kaggle.json to kaggle.json
ref                                                     title                                                   size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------------  -----------------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
awsaf49/cbis-ddsm-breast-cancer-image-dataset           CBIS-DDSM: Breast Cancer Image Dataset            5318997088  2021-01-24 07:35:59.737000          45843        331  0.7058824        
skooch/ddsm-mammography                                 DDSM Mammography                                  3093452937  2018-07-03 08:38:28.307000          10668        193  0.75             
llkihn/ddsm-cbis-patch                                  DDSM CBIS Patch                                   1276311012  2021-01-02 23:19:16.743000            248          7  1.0              


In [ ]:
from google.colab import drive

drive.mount("/content/drive")

BASE_DIR = "/content/drive/MyDrive/mammo_cad"
ZIP_DIR = f"{BASE_DIR}/zips"
RAW_DIR = f"{BASE_DIR}/raw"

import os

os.makedirs(ZIP_DIR, exist_ok=True)
os.makedirs(RAW_DIR, exist_ok=True)

print("BASE_DIR:", BASE_DIR)
print("ZIP_DIR :", ZIP_DIR)
print("RAW_DIR :", RAW_DIR)

Mounted at /content/drive
BASE_DIR: /content/drive/MyDrive/mammo_cad
ZIP_DIR : /content/drive/MyDrive/mammo_cad/zips
RAW_DIR : /content/drive/MyDrive/mammo_cad/raw


In [ ]:
import glob

import os
import zipfile

ZIP_DIR = "/content/drive/MyDrive/mammo_cad/zips"
RAW_DIR = "/content/drive/MyDrive/mammo_cad/raw"

zips_before = glob.glob(ZIP_DIR + "/*.zip")
print("Zips already in Drive:", len(zips_before))

if len(zips_before) == 0:

    !kaggle datasets download -d awsaf49/cbis-ddsm-breast-cancer-image-dataset -p "$ZIP_DIR"
else:
    print("⏭️ Zip already present, skipping download.")

os.makedirs(RAW_DIR, exist_ok=True)
inner_any = glob.glob(RAW_DIR + "/**/*", recursive=True)
print("Files already in RAW_DIR:", len(inner_any))

if len(inner_any) < 100:
    for z in glob.glob(ZIP_DIR + "/*.zip"):
        print("Extracting:", os.path.basename(z))
        with zipfile.ZipFile(z, "r") as zf:
            zf.extractall(RAW_DIR)
else:
    print("⏭️ RAW_DIR already populated, skipping extract.")

import glob

jpgs = glob.glob(f"{RAW_DIR}/**/*.jpg", recursive=True) + glob.glob(
    f"{RAW_DIR}/**/*.jpeg", recursive=True
)
pngs = glob.glob(f"{RAW_DIR}/**/*.png", recursive=True)
csvs = glob.glob(f"{RAW_DIR}/**/*.csv", recursive=True)

print("Images -> JPG/JPEG:", len(jpgs), " PNG:", len(pngs))
print("CSV files:", len(csvs))
print("Sample image:", (jpgs + pngs)[:2])
print("Sample CSV:", csvs[:3])

Zips already in Drive: 0
Dataset URL: https://www.kaggle.com/datasets/awsaf49/cbis-ddsm-breast-cancer-image-dataset
License(s): CC-BY-SA-3.0
100% 4.95G/4.95G [00:34<00:00, 190MB/s]
100% 4.95G/4.95G [00:34<00:00, 152MB/s]
Files already in RAW_DIR: 0
Extracting: cbis-ddsm-breast-cancer-image-dataset.zip
Images -> JPG/JPEG: 10237  PNG: 0
CSV files: 6
Sample image: ['/content/drive/MyDrive/mammo_cad/raw/jpeg/1.3.6.1.4.1.9590.100.1.2.100018879311824535125115145152454291132/1-263.jpg', '/content/drive/MyDrive/mammo_cad/raw/jpeg/1.3.6.1.4.1.9590.100.1.2.100018879311824535125115145152454291132/2-241.jpg']
Sample CSV: ['/content/drive/MyDrive/mammo_cad/raw/csv/calc_case_description_test_set.csv', '/content/drive/MyDrive/mammo_cad/raw/csv/calc_case_description_train_set.csv', '/content/drive/MyDrive/mammo_cad/raw/csv/dicom_info.csv']


In [ ]:
import glob
import re

import pandas as pd

RAW_DIR = "/content/drive/MyDrive/mammo_cad/raw"
UID_RE = re.compile(r"(1(?:\.\d+){3,})")


def extract_uid(s):
    m = UID_RE.search(str(s))
    return m.group(1) if m else None


# 3.1) Mass CSVs
mass_csvs = glob.glob(f"{RAW_DIR}/csv/mass_case_description_*_set.csv")
print("Mass CSVs:", mass_csvs)

mass_uids = set()
mass_pid_by_uid = {}
for c in mass_csvs:
    df = pd.read_csv(c)
    for _, row in df.iterrows():
        uid = extract_uid(row.get("image file path", ""))
        if uid:
            mass_uids.add(uid)
            pid = str(row.get("patient_id", "")).strip().lower()
            if pid:
                mass_pid_by_uid[uid] = pid

print("Total Mass UIDs:", len(mass_uids))
print("Sample Mass UIDs:", list(mass_uids)[:5])

# 3.2) Calc CSVs (فقط برای اینکه کنار بگذاریم‌شان)
calc_csvs = glob.glob(f"{RAW_DIR}/csv/calc_case_description_*_set.csv")
calc_uids = set()
for c in calc_csvs:
    df = pd.read_csv(c)
    for _, row in df.iterrows():
        uid = extract_uid(row.get("image file path", ""))
        if uid:
            calc_uids.add(uid)

print("Total Calc UIDs:", len(calc_uids))
print("Overlap Mass∩Calc:", len(mass_uids & calc_uids))

Mass CSVs: ['/content/drive/MyDrive/mammo_cad/raw/csv/mass_case_description_test_set.csv', '/content/drive/MyDrive/mammo_cad/raw/csv/mass_case_description_train_set.csv']
Total Mass UIDs: 1592
Sample Mass UIDs: ['1.3.6.1.4.1.9590.100.1.2.244860754512847405721118721642884345502', '1.3.6.1.4.1.9590.100.1.2.143651499712850251702614043240502055735', '1.3.6.1.4.1.9590.100.1.2.236750089410442340428255082793184632940', '1.3.6.1.4.1.9590.100.1.2.96699930111611146740318739371000200310', '1.3.6.1.4.1.9590.100.1.2.401772036413964151122239013710383266724']
Total Calc UIDs: 1511
Overlap Mass∩Calc: 0


In [ ]:
import glob

import os
import re

import pandas as pd

RAW_DIR = "/content/drive/MyDrive/mammo_cad/raw"
CSV_DIR = f"{RAW_DIR}/csv"
SRC_IMG_DIR = f"{RAW_DIR}/jpeg"

UID_RE = re.compile(r"(1(?:\.\d+){3,})")


def extract_uid(s):
    m = UID_RE.search(str(s))
    return m.group(1) if m else None



mass_uids, mass_pid_by_uid = set(), {}
mass_csvs = glob.glob(f"{CSV_DIR}/mass_case_description_*_set.csv")
for c in mass_csvs:
    df = pd.read_csv(c)
    for _, row in df.iterrows():
        u = extract_uid(row.get("image file path", ""))
        if u:
            mass_uids.add(u)
            pid = str(row.get("patient_id", "")).strip().lower()
            if pid:
                mass_pid_by_uid[u] = pid


calc_uids = set()
calc_csvs = glob.glob(f"{CSV_DIR}/calc_case_description_*_set.csv")
for c in calc_csvs:
    df = pd.read_csv(c)
    for _, row in df.iterrows():
        u = extract_uid(row.get("image file path", ""))
        if u:
            calc_uids.add(u)

# --- 3) UIDهای موجود در مسیر تصاویر jpeg ---
jpeg_files = (
    glob.glob(f"{SRC_IMG_DIR}/**/*.jpg", recursive=True)
    + glob.glob(f"{SRC_IMG_DIR}/**/*.jpeg", recursive=True)
    + glob.glob(f"{SRC_IMG_DIR}/**/*.png", recursive=True)
)
jpeg_uids = {extract_uid(p) for p in jpeg_files if extract_uid(p)}

print(
    "Counts -> mass_uids(csv):",
    len(mass_uids),
    "| calc_uids(csv):",
    len(calc_uids),
    "| jpeg_uids(files):",
    len(jpeg_uids),
)
print("Intersection(jpeg ∩ mass):", len(jpeg_uids & mass_uids))
print("Intersection(jpeg ∩ calc):", len(jpeg_uids & calc_uids))


# چند نمونه برای sanity-check
def sample(s, k=3):
    s = list(s)
    return s[: min(k, len(s))]


print("Sample mass UID (csv):", sample(mass_uids))
print("Sample jpeg UID (files):", sample(jpeg_uids))

Counts -> mass_uids(csv): 1592 | calc_uids(csv): 1511 | jpeg_uids(files): 6774
Intersection(jpeg ∩ mass): 0
Intersection(jpeg ∩ calc): 0
Sample mass UID (csv): ['1.3.6.1.4.1.9590.100.1.2.244860754512847405721118721642884345502', '1.3.6.1.4.1.9590.100.1.2.143651499712850251702614043240502055735', '1.3.6.1.4.1.9590.100.1.2.236750089410442340428255082793184632940']
Sample jpeg UID (files): ['1.3.6.1.4.1.9590.100.1.2.259575017012299394931844213051161127499', '1.3.6.1.4.1.9590.100.1.2.306231427610466354735096101303712317796', '1.3.6.1.4.1.9590.100.1.2.291970834411466767515548699420271393213']


In [ ]:
import glob
import re

# مرحله ۴-B: ساخت نگاشت از dicom_info.csv و ترجمهٔ UIDهای mass به jpeg
import pandas as pd

RAW_DIR = "/content/drive/MyDrive/mammo_cad/raw"
CSV_DIR = f"{RAW_DIR}/csv"
UID_RE = re.compile(r"(1(?:\.\d+){3,})")


def extract_uid(s):
    m = UID_RE.search(str(s))
    return m.group(1) if m else None


# 1) خواندن dicom_info و ساخت دیکشنری dicom→jpeg
df_info = pd.read_csv(f"{CSV_DIR}/dicom_info.csv")
dicom_uid = df_info["file_path"].apply(extract_uid)
jpeg_uid = df_info["image_path"].apply(extract_uid)
dicom2jpeg = {d: j for d, j in zip(dicom_uid, jpeg_uid) if pd.notna(d) and pd.notna(j)}
print("dicom2jpeg size:", len(dicom2jpeg))

# 2) اگر mass_uids/calc_uids از بخش A در حافظه هست از همان‌ها استفاده می‌کنیم؛
#    اگر نبود دوباره سریع می‌سازیم.
import os

if "mass_uids" not in globals() or "calc_uids" not in globals():
    mass_uids, mass_pid_by_uid = set(), {}
    for c in glob.glob(f"{CSV_DIR}/mass_case_description_*_set.csv"):
        df = pd.read_csv(c)
        for _, row in df.iterrows():
            u = extract_uid(row.get("image file path", ""))
            if u:
                mass_uids.add(u)
                pid = str(row.get("patient_id", "")).strip().lower()
                if pid:
                    mass_pid_by_uid[u] = pid
    calc_uids = set()
    for c in glob.glob(f"{CSV_DIR}/calc_case_description_*_set.csv"):
        df = pd.read_csv(c)
        for _, row in df.iterrows():
            u = extract_uid(row.get("image file path", ""))
            if u:
                calc_uids.add(u)

# 3) ترجمهٔ UIDهای mass از فضای DICOM به فضای JPEG
mass_uids_jpeg = {dicom2jpeg[u] for u in mass_uids if u in dicom2jpeg}
print("mass_uids (dicom):", len(mass_uids), "→ mass_uids_jpeg:", len(mass_uids_jpeg))

# 4) هم‌پوشانی جدید با UIDهای فایل‌های jpeg
import glob

SRC_IMG_DIR = f"{RAW_DIR}/jpeg"
jpeg_files = glob.glob(f"{SRC_IMG_DIR}/**/*.jpg", recursive=True) + glob.glob(
    f"{SRC_IMG_DIR}/**/*.jpeg", recursive=True
)
jpeg_uids = {extract_uid(p) for p in jpeg_files if extract_uid(p)}
print("jpeg_uids:", len(jpeg_uids))
print("Intersection(jpeg ∩ mass_jpeg):", len(jpeg_uids & mass_uids_jpeg))

# 5) چند نمونه برای sanity-check
print("Sample mass_jpeg UID:", list(mass_uids_jpeg)[:5])

dicom2jpeg size: 6774
mass_uids (dicom): 1592 → mass_uids_jpeg: 0
jpeg_uids: 6774
Intersection(jpeg ∩ mass_jpeg): 0
Sample mass_jpeg UID: []


In [ ]:
import glob
import re

# 4-C: پیدا کردن این‌که کدام UID در dicom_info با پوشه‌های jpeg/ یکی است
import pandas as pd

RAW_DIR = "/content/drive/MyDrive/mammo_cad/raw"
CSV_DIR = f"{RAW_DIR}/csv"
UID_RE = re.compile(r"(1(?:\.\d+){3,})")


def extract_uid(s):
    m = UID_RE.search(str(s))
    return m.group(1) if m else None


# 1) فایل dicom_info را بخوان
info = pd.read_csv(f"{CSV_DIR}/dicom_info.csv")

# 2) از ستون‌های مختلف، UID استخراج کن
uid_from_file_path = info.get("file_path", "").apply(extract_uid)
uid_from_image_path = info.get("image_path", "").apply(extract_uid)
uid_series = info.get("SeriesInstanceUID", "").apply(extract_uid)
uid_study = info.get("StudyInstanceUID", "").apply(extract_uid)

set_file = set(x for x in uid_from_file_path.dropna().tolist() if isinstance(x, str))
set_image = set(x for x in uid_from_image_path.dropna().tolist() if isinstance(x, str))
set_series = set(x for x in uid_series.dropna().tolist() if isinstance(x, str))
set_study = set(x for x in uid_study.dropna().tolist() if isinstance(x, str))

print(
    "Counts in dicom_info -> file:",
    len(set_file),
    "| image:",
    len(set_image),
    "| series:",
    len(set_series),
    "| study:",
    len(set_study),
)

# 3) UIDهای پوشه‌های jpeg/ (از قبل)
import glob

jpeg_uids = set()
for p in glob.glob(f"{RAW_DIR}/jpeg/**/*.jpg", recursive=True):
    u = extract_uid(p)
    if u:
        jpeg_uids.add(u)

print("jpeg_uids(files):", len(jpeg_uids))

# 4) هم‌پوشانی با هر ستون:
print("Overlap jpeg∩file  :", len(jpeg_uids & set_file))
print("Overlap jpeg∩image :", len(jpeg_uids & set_image))
print("Overlap jpeg∩series:", len(jpeg_uids & set_series))
print("Overlap jpeg∩study :", len(jpeg_uids & set_study))

# 5) UIDهای mass را از هر دو ستونِ mass-CSV می‌گیریم (image file path و cropped image file path)
mass_csvs = glob.glob(f"{CSV_DIR}/mass_case_description_*_set.csv")
mass_uids_image = set()
mass_uids_cropped = set()
mass_pid_by_uid = {}
for c in mass_csvs:
    df = pd.read_csv(c)
    for _, row in df.iterrows():
        u1 = extract_uid(row.get("image file path", ""))
        u2 = extract_uid(row.get("cropped image file path", ""))
        if u1:
            mass_uids_image.add(u1)
            pid = str(row.get("patient_id", "")).strip().lower()
            if pid:
                mass_pid_by_uid[u1] = pid
        if u2:
            mass_uids_cropped.add(u2)
            pid = str(row.get("patient_id", "")).strip().lower()
            if pid and u2 not in mass_pid_by_uid:
                mass_pid_by_uid[u2] = pid

print("mass_uids_image  :", len(mass_uids_image))
print("mass_uids_cropped:", len(mass_uids_cropped))

# 6) ببینیم کدام‌یک از mass_uids_* با کدام ستون از dicom_info می‌نشیند
print("Overlap mass_image ∩ file  :", len(mass_uids_image & set_file))
print("Overlap mass_image ∩ image :", len(mass_uids_image & set_image))
print("Overlap mass_image ∩ series:", len(mass_uids_image & set_series))
print("Overlap mass_image ∩ study :", len(mass_uids_image & set_study))

print("Overlap mass_crop  ∩ file  :", len(mass_uids_cropped & set_file))
print("Overlap mass_crop  ∩ image :", len(mass_uids_cropped & set_image))
print("Overlap mass_crop  ∩ series:", len(mass_uids_cropped & set_series))
print("Overlap mass_crop  ∩ study :", len(mass_uids_cropped & set_study))


# 7) چند UID نمونه از هر طرف برای sanity check
def sample(s, k=3):
    s = list(s)
    return s[: min(k, len(s))]


print("Sample jpeg_uids:", sample(jpeg_uids))
print("Sample set_image:", sample(set_image))
print("Sample set_series:", sample(set_series))
print("Sample mass_uids_image:", sample(mass_uids_image))
print("Sample mass_uids_cropped:", sample(mass_uids_cropped))

Counts in dicom_info -> file: 6774 | image: 6774 | series: 6774 | study: 6774
jpeg_uids(files): 6774
Overlap jpeg∩file  : 6774
Overlap jpeg∩image : 6774
Overlap jpeg∩series: 6774
Overlap jpeg∩study : 0
mass_uids_image  : 1592
mass_uids_cropped: 1696
Overlap mass_image ∩ file  : 0
Overlap mass_image ∩ image : 0
Overlap mass_image ∩ series: 0
Overlap mass_image ∩ study : 1592
Overlap mass_crop  ∩ file  : 0
Overlap mass_crop  ∩ image : 0
Overlap mass_crop  ∩ series: 0
Overlap mass_crop  ∩ study : 1696
Sample jpeg_uids: ['1.3.6.1.4.1.9590.100.1.2.259575017012299394931844213051161127499', '1.3.6.1.4.1.9590.100.1.2.306231427610466354735096101303712317796', '1.3.6.1.4.1.9590.100.1.2.291970834411466767515548699420271393213']
Sample set_image: ['1.3.6.1.4.1.9590.100.1.2.306231427610466354735096101303712317796', '1.3.6.1.4.1.9590.100.1.2.259575017012299394931844213051161127499', '1.3.6.1.4.1.9590.100.1.2.291970834411466767515548699420271393213']
Sample set_series: ['1.3.6.1.4.1.9590.100.1.2.3062

In [ ]:
import glob
import re

# 4-D: ترجمه‌ی StudyInstanceUIDهای Mass به مجموعه‌ای از Image UIDها با dicom_info.csv
import pandas as pd

RAW_DIR = "/content/drive/MyDrive/mammo_cad/raw"
CSV_DIR = f"{RAW_DIR}/csv"

UID_RE = re.compile(r"(1(?:\.\d+){3,})")


def extract_uid(s):
    m = UID_RE.search(str(s))
    return m.group(1) if m else None


# 1) Mass studies + pid map از CSVهای رسمی
mass_csvs = glob.glob(f"{CSV_DIR}/mass_case_description_*_set.csv")
mass_study_uids = set()
mass_pid_by_study = {}

for c in mass_csvs:
    df = pd.read_csv(c)
    for _, row in df.iterrows():
        # طبق یافته‌ی قبلی، 'image file path' و 'cropped image file path' حاوی StudyInstanceUID هستند
        for col in ["image file path", "cropped image file path"]:
            u = extract_uid(row.get(col, ""))
            if u:
                mass_study_uids.add(u)
                pid = str(row.get("patient_id", "")).strip().lower()
                if pid and u not in mass_pid_by_study:
                    mass_pid_by_study[u] = pid

print("Mass Study UIDs:", len(mass_study_uids))

# 2) dicom_info: نگاشت Study -> ImageUID و برعکس
info = pd.read_csv(f"{CSV_DIR}/dicom_info.csv")
study_uid_col = info.get("StudyInstanceUID", "").apply(extract_uid)
image_uid_col = info.get("image_path", "").apply(extract_uid)

study2images = {}
image2study = {}
for s_uid, i_uid in zip(study_uid_col, image_uid_col):
    if isinstance(s_uid, str) and isinstance(i_uid, str):
        study2images.setdefault(s_uid, set()).add(i_uid)
        image2study[i_uid] = s_uid

print("study2images keys:", len(study2images), "| image2study:", len(image2study))

# 3) تبدیل Mass Study → مجموعه‌ی Image UIDها در jpeg
mass_uids_jpeg = set()
for s in mass_study_uids:
    if s in study2images:
        mass_uids_jpeg.update(study2images[s])

print("mass_uids_jpeg (translated):", len(mass_uids_jpeg))

# 4) هم‌پوشانی با UIDهای واقعی فایل‌های jpeg
import glob

jpeg_files = glob.glob(f"{RAW_DIR}/jpeg/**/*.jpg", recursive=True) + glob.glob(
    f"{RAW_DIR}/jpeg/**/*.jpeg", recursive=True
)
jpeg_uids = {extract_uid(p) for p in jpeg_files if extract_uid(p)}

print("jpeg_uids:", len(jpeg_uids))
print("Intersection(jpeg ∩ mass_uids_jpeg):", len(jpeg_uids & mass_uids_jpeg))


# 5) چند نمونه برای sanity check
def sample(s, k=5):
    s = list(s)
    return s[: min(k, len(s))]


print("Sample mass_study_uids:", sample(mass_study_uids))
print("Sample mass_uids_jpeg :", sample(mass_uids_jpeg))

Mass Study UIDs: 3288
study2images keys: 6774 | image2study: 6774
mass_uids_jpeg (translated): 3288
jpeg_uids: 6774
Intersection(jpeg ∩ mass_uids_jpeg): 3288
Sample mass_study_uids: ['1.3.6.1.4.1.9590.100.1.2.244860754512847405721118721642884345502', '1.3.6.1.4.1.9590.100.1.2.143651499712850251702614043240502055735', '1.3.6.1.4.1.9590.100.1.2.236750089410442340428255082793184632940', '1.3.6.1.4.1.9590.100.1.2.128382012316476040430900783769217916', '1.3.6.1.4.1.9590.100.1.2.282915215111615665204682464000406265581']
Sample mass_uids_jpeg : ['1.3.6.1.4.1.9590.100.1.2.165978598313546365331998685770369073828', '1.3.6.1.4.1.9590.100.1.2.285317105313152046222309992052850234643', '1.3.6.1.4.1.9590.100.1.2.306231427610466354735096101303712317796', '1.3.6.1.4.1.9590.100.1.2.345919480011577581036244773132668941332', '1.3.6.1.4.1.9590.100.1.2.305524546111378534235330467170895252659']


In [ ]:
import glob
import hashlib

# مرحله ۴-E: build patient-level split & copy
import os
import pathlib
import random
import re
import shutil
from collections import defaultdict

import pandas as pd

RAW_DIR = "/content/drive/MyDrive/mammo_cad/raw"
CSV_DIR = f"{RAW_DIR}/csv"
SRC_IMG_DIR = f"{RAW_DIR}/jpeg"
DEST_ROOT = "/content/data"

UID_RE = re.compile(r"(1(?:\.\d+){3,})")


def extract_uid(s):
    m = UID_RE.search(str(s))
    return m.group(1) if m else None


# 1) از بخش D: این‌ها باید در حافظه باشند:
# - mass_uids_jpeg  (Image UIDهایی که Mass هستند)
# - image2study     (Image UID -> Study UID)
# - mass_pid_by_study (Study UID -> patient_id)
assert "mass_uids_jpeg" in globals()
assert "image2study" in globals()
assert "mass_pid_by_study" in globals()

# 2) ترجمه‌ی calc به Image UID برای حذف
calc_study_uids = set()
for c in glob.glob(f"{CSV_DIR}/calc_case_description_*_set.csv"):
    df = pd.read_csv(c)
    for _, row in df.iterrows():
        u = extract_uid(row.get("image file path", ""))
        if u:
            calc_study_uids.add(u)

# study2images را از image2study می‌سازیم
from collections import defaultdict as dd

study2images = dd(set)
for img_uid, stu_uid in image2study.items():
    if isinstance(stu_uid, str) and isinstance(img_uid, str):
        study2images[stu_uid].add(img_uid)

calc_uids_jpeg = set()
for s in calc_study_uids:
    if s in study2images:
        calc_uids_jpeg.update(study2images[s])

print(
    "calc_study_uids:", len(calc_study_uids), "-> calc_uids_jpeg:", len(calc_uids_jpeg)
)

# 3) جمع‌آوری همه تصاویر و ساخت برچسب + pid
all_imgs = glob.glob(f"{SRC_IMG_DIR}/**/*.jpg", recursive=True) + glob.glob(
    f"{SRC_IMG_DIR}/**/*.jpeg", recursive=True
)


def uid_from_path(p):
    m = UID_RE.search(p)
    return m.group(1) if m else None


def pid_from_path(p):
    parent = pathlib.Path(p).parent.as_posix().lower()
    return "pid_" + hashlib.md5(parent.encode()).hexdigest()[:10]


by_patient = dd(list)

for p in all_imgs:
    img_uid = uid_from_path(p)
    if not img_uid:
        continue

    if img_uid in calc_uids_jpeg:
        continue

    if img_uid in mass_uids_jpeg:
        lab = "mass"
        stu = image2study.get(img_uid, None)
        if stu and stu in mass_pid_by_study:
            pid = "csv_" + mass_pid_by_study[stu]
        else:
            pid = pid_from_path(p)
    else:
        lab = "normal"
        pid = pid_from_path(p)
    by_patient[pid].append((p, lab))

print("Patients total:", len(by_patient))

# 4) split بیمار-محور 70/15/15
patients = list(by_patient.keys())
random.seed(42)
random.shuffle(patients)
n = len(patients)
pt_train = set(patients[: int(0.7 * n)])
pt_val = set(patients[int(0.7 * n) : int(0.85 * n)])
pt_test = set(patients[int(0.85 * n) :])

# 5) کپی با سقف حجم
for split in ["train", "val", "test"]:
    for cls in ["mass", "normal"]:
        d = f"{DEST_ROOT}/{split}/{cls}"
        if os.path.exists(d):
            shutil.rmtree(d, ignore_errors=True)
        os.makedirs(d, exist_ok=True)

CAP_PER_CLASS = 800  # اگر لازم بود 600/400 کن
counters = {
    ("train", "mass"): 0,
    ("train", "normal"): 0,
    ("val", "mass"): 0,
    ("val", "normal"): 0,
    ("test", "mass"): 0,
    ("test", "normal"): 0,
}


def copy_group(pids, split):
    for pid in pids:
        for src, lab in by_patient[pid]:
            key = (split, lab)
            if counters[key] >= CAP_PER_CLASS:
                continue
            dst = f"{DEST_ROOT}/{split}/{lab}/{pid}__{os.path.basename(src)}"
            shutil.copy2(src, dst)
            counters[key] += 1


copy_group(pt_train, "train")
copy_group(pt_val, "val")
copy_group(pt_test, "test")


def count_in(folder):
    return len(glob.glob(folder + "/*"))


summary = {
    "train": {
        "mass": count_in(f"{DEST_ROOT}/train/mass"),
        "normal": count_in(f"{DEST_ROOT}/train/normal"),
    },
    "val": {
        "mass": count_in(f"{DEST_ROOT}/val/mass"),
        "normal": count_in(f"{DEST_ROOT}/val/normal"),
    },
    "test": {
        "mass": count_in(f"{DEST_ROOT}/test/mass"),
        "normal": count_in(f"{DEST_ROOT}/test/normal"),
    },
}
summary

calc_study_uids: 1511 -> calc_uids_jpeg: 1511
Patients total: 2867


{'train': {'mass': 799, 'normal': 800},
 'val': {'mass': 759, 'normal': 550},
 'test': {'mass': 797, 'normal': 576}}

In [ ]:

%%writefile /content/dataset.py
import glob
import os

import torch
from PIL import Image
from torch.utils.data import Dataset


class MammogramFolders(Dataset):
    """
    Expect: root/{mass,normal}/*.jpg|*.png
    - تصاویر تک‌کاناله هستند؛ ولی برای مدل‌های ImageNet آن را به 3ch تبدیل می‌کنیم (در transforms).
    - برچسب: normal=0, mass=1
    """

    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.samples = []
        for label_name in ["normal", "mass"]:
            label = 0 if label_name == "normal" else 1
            for p in glob.glob(os.path.join(root, label_name, "*")):
                if p.lower().endswith(
                    (".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff")
                ):
                    self.samples.append((p, label))
        if len(self.samples) == 0:
            raise RuntimeError(
                f"No images found under {root}. Expected 'mass' and 'normal' subfolders."
            )

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("L")
        if self.transform:
            img = self.transform(img)  # اینجا به 3ch تبدیل می‌کنیم
        return img, torch.tensor(label, dtype=torch.float32)

    def pos_weight(self):
        n_pos = sum(1 for _, y in self.samples if y == 1)
        n_neg = sum(1 for _, y in self.samples if y == 0)
        return (n_neg / max(1, n_pos)) if n_pos > 0 else 1.0

Writing /content/dataset.py


In [ ]:

import torch
from dataset import MammogramFolders
from torch.utils.data import DataLoader
from torchvision import transforms

DATA_ROOT = "/content/data"

tfm = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),  # gray → 3 کانال
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ]
)

ds_train = MammogramFolders(f"{DATA_ROOT}/train", transform=tfm)
dl_train = DataLoader(
    ds_train, batch_size=4, shuffle=True, num_workers=2, pin_memory=True
)

print("Train size:", len(ds_train), "| pos_weight≈", ds_train.pos_weight())

x, y = next(iter(dl_train))
print("Batch shape:", x.shape, "| labels:", y)
print("Min/Max:", float(x.min()), float(x.max()))

Train size: 1599 | pos_weight≈ 1.0012515644555695
Batch shape: torch.Size([4, 3, 512, 512]) | labels: tensor([0., 0., 0., 1.])
Min/Max: 0.0 1.0


In [ ]:
%%writefile /content/model.py
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18

class LiteAttention(nn.Module):
    """
    یک بلاک خیلی سبک برای توجه مکانی (spatial) + کانالی (channel) با محاسبه‌ی ارزان.
    - کانالی: squeeze-excitation کوچک (r=8)
    - مکانی: average + max pooling کانال‌ها و یک conv 7x7 کم‌هزینه
    """
    def __init__(self, c, r=8):
        super().__init__()
        r = max(1, c // r)
        # Channel attention (SE)
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(c, r, kernel_size=1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(r, c, kernel_size=1, bias=False),
            nn.Sigmoid()
        )
        # Spatial attention
        self.spatial = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=7, padding=3, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Channel attention
        w_c = self.se(x)
        x = x * w_c
        # Spatial attention
        avg_map = torch.mean(x, dim=1, keepdim=True)
        max_map, _ = torch.max(x, dim=1, keepdim=True)
        s = torch.cat([avg_map, max_map], dim=1)
        w_s = self.spatial(s)
        return x * w_s

class ResNet18LiteAtt(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        # backbone
        self.backbone = resnet18(weights="IMAGENET1K_V1" if pretrained else None)
        # حذف avgpool و fc تا ویژگی‌ها را خودمان جمع‌بندی کنیم
        self.features = nn.Sequential(
            self.backbone.conv1,
            self.backbone.bn1,
            self.backbone.relu,
            self.backbone.maxpool,

            self.backbone.layer1,
            self.backbone.layer2,
            self.backbone.layer3,
            self.backbone.layer4
        )
        c = 512  # خروجی resnet18
        self.attn = LiteAttention(c, r=8)
        # سرِ طبقه‌بندی: global pooling دوتایی + head کم‌پارامتر
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(c, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        # ورودی باید [B,3,H,W] باشد (ما قبلش به 3ch تبدیل می‌کنیم)
        f = self.features(x)          # [B, 512, H/32, W/32]
        f = self.attn(f)              # LiteAttention
        out = self.head(f)            # [B, 1]
        return out.squeeze(1)         # [B]

def build_model(pretrained=True):
    return ResNet18LiteAtt(pretrained=pretrained)

Overwriting /content/model.py


In [ ]:
import torch
from model import build_model

m = build_model(pretrained=True).cuda()
x = torch.randn(2, 3, 512, 512).cuda()
with torch.no_grad():
    y = m(x)
print("Model OK | logits shape:", y.shape)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 127MB/s]


Model OK | logits shape: torch.Size([2])


In [ ]:
import math

# Supervised CNN training (Smoke Test: 2 epochs)
import os
import time

import numpy as np
import torch
from dataset import MammogramFolders
from model import build_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader
from torchvision import transforms

DATA_ROOT = "/content/data"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

#  (train/val/test)
tfm_train = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=5),
        transforms.ToTensor(),
    ]
)
tfm_eval = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ]
)


ds_train = MammogramFolders(f"{DATA_ROOT}/train", transform=tfm_train)
ds_val = MammogramFolders(f"{DATA_ROOT}/val", transform=tfm_eval)
ds_test = MammogramFolders(f"{DATA_ROOT}/test", transform=tfm_eval)

pos_weight = torch.tensor([ds_train.pos_weight()], device=DEVICE)
print(
    f"Train size={len(ds_train)} | Val size={len(ds_val)} | Test size={len(ds_test)} | pos_weight≈{pos_weight.item():.3f}"
)

dl_train = DataLoader(
    ds_train, batch_size=16, shuffle=True, num_workers=2, pin_memory=True
)
dl_val = DataLoader(
    ds_val, batch_size=32, shuffle=False, num_workers=2, pin_memory=True
)


model = build_model(pretrained=True).to(DEVICE)
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)  # برای کنترل عدم‌توازن
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2)

scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE == "cuda"))


def evaluate(model, loader):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(DEVICE, non_blocking=True)
            y = y.to(DEVICE)
            logits = model(x)
            psig = torch.sigmoid(logits)
            ys.append(y.float().cpu().numpy())
            ps.append(psig.float().cpu().numpy())
    y_true = np.concatenate(ys)
    y_prob = np.concatenate(ps)
    y_pred = (y_prob >= 0.5).astype(np.float32)

    try:
        auc = roc_auc_score(y_true, y_prob)
    except Exception:
        auc = float("nan")
    f1 = f1_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    tn, fp, fn, tf = confusion_matrix(y_true, y_pred).ravel()
    sen = tf / (tf + fn + 1e-8)  # sensitivity/recall for positive class
    spe = tn / (tn + fp + 1e-8)  # specificity

    return {"AUC": auc, "F1": f1, "ACC": acc, "SEN": sen, "SPE": spe}


EPOCHS = 2
best_auc = -1.0
os.makedirs("/content/runs", exist_ok=True)
ckpt_path = "/content/runs/best.pt"

for epoch in range(1, EPOCHS + 1):
    model.train()
    running = 0.0
    n = 0
    t0 = time.time()
    for x, y in dl_train:
        x = x.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE)

import math


import os
import time

import numpy as np
import torch
from dataset import MammogramFolders
from model import build_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader
from torchvision import transforms

DATA_ROOT = "/content/data"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- Transforms (train/val/test) ---
tfm_train = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=5),
        transforms.ToTensor(),
    ]
)
tfm_eval = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ]
)


ds_train = MammogramFolders(f"{DATA_ROOT}/train", transform=tfm_train)
ds_val = MammogramFolders(f"{DATA_ROOT}/val", transform=tfm_eval)
ds_test = MammogramFolders(f"{DATA_ROOT}/test", transform=tfm_eval)

pos_weight = torch.tensor([ds_train.pos_weight()], device=DEVICE)
print(
    f"Train size={len(ds_train)} | Val size={len(ds_val)} | Test size={len(ds_test)} | pos_weight≈{pos_weight.item():.3f}"
)

dl_train = DataLoader(
    ds_train, batch_size=16, shuffle=True, num_workers=2, pin_memory=True
)
dl_val = DataLoader(
    ds_val, batch_size=32, shuffle=False, num_workers=2, pin_memory=True
)

# --- Model / Loss / Optimizer ---
model = build_model(pretrained=True).to(DEVICE)
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)  # برای کنترل عدم‌توازن
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2)

scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE == "cuda"))


def evaluate(model, loader):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(DEVICE, non_blocking=True)
            y = y.to(DEVICE)
            logits = model(x)
            psig = torch.sigmoid(logits)
            ys.append(y.float().cpu().numpy())
            ps.append(psig.float().cpu().numpy())
    y_true = np.concatenate(ys)
    y_prob = np.concatenate(ps)
    y_pred = (y_prob >= 0.5).astype(np.float32)

    try:
        auc = roc_auc_score(y_true, y_prob)
    except Exception:
        auc = float("nan")
    f1 = f1_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    tn, fp, fn, tf = confusion_matrix(y_true, y_pred).ravel()
    sen = tf / (tf + fn + 1e-8)  # sensitivity/recall for positive class
    spe = tn / (tn + fp + 1e-8)  # specificity

    return {"AUC": auc, "F1": f1, "ACC": acc, "SEN": sen, "SPE": spe}


# Train (2 epochs smoke test)
EPOCHS = 2
best_auc = -1.0
os.makedirs("/content/runs", exist_ok=True)
ckpt_path = "/content/runs/best.pt"

for epoch in range(1, EPOCHS + 1):
    model.train()
    running = 0.0
    n = 0
    t0 = time.time()
    for x, y in dl_train:
        x = x.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE)

Train size=1599 | Val size=1309 | Test size=1373 | pos_weight≈1.001


/tmp/ipython-input-3964383616.py:44: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))


Train size=1599 | Val size=1309 | Test size=1373 | pos_weight≈1.001


/tmp/ipython-input-3964383616.py:131: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))


In [ ]:
import math

#  Supervised CNN training (Smoke Test: 2 epochs, AMP new API)
import os
import random
import time

import numpy as np
import torch
from dataset import MammogramFolders
from model import build_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader
from torchvision import transforms

# reproducibility سبک
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

DATA_ROOT = "/content/data"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


tfm_train = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=5),
        transforms.ToTensor(),
    ]
)
tfm_eval = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ]
)


ds_train = MammogramFolders(f"{DATA_ROOT}/train", transform=tfm_train)
ds_val = MammogramFolders(f"{DATA_ROOT}/val", transform=tfm_eval)
ds_test = MammogramFolders(f"{DATA_ROOT}/test", transform=tfm_eval)

print(
    f"Train size={len(ds_train)} | Val size={len(ds_val)} | Test size={len(ds_test)} | pos_weight≈{ds_train.pos_weight():.3f}"
)

dl_train = DataLoader(
    ds_train, batch_size=16, shuffle=True, num_workers=2, pin_memory=True
)
dl_val = DataLoader(
    ds_val, batch_size=32, shuffle=False, num_workers=2, pin_memory=True
)

#  Model / Loss / Optimizer
model = build_model(pretrained=True).to(DEVICE)
pos_weight = torch.tensor([ds_train.pos_weight()], device=DEVICE)
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2)

# ✅ AMP جدید
from torch.amp import GradScaler
from torch.amp import autocast

scaler = GradScaler(device=DEVICE if DEVICE == "cuda" else "cpu")


def evaluate(model, loader):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(DEVICE, non_blocking=True)
            y = y.to(DEVICE)
            logits = model(x)
            psig = torch.sigmoid(logits)
            ys.append(y.float().cpu().numpy())
            ps.append(psig.float().cpu().numpy())
    y_true = np.concatenate(ys)
    y_prob = np.concatenate(ps)
    y_pred = (y_prob >= 0.5).astype(np.float32)

    try:
        auc = roc_auc_score(y_true, y_prob)
    except Exception:
        auc = float("nan")
    f1 = f1_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sen = tp / (tp + fn + 1e-8)
    spe = tn / (tn + fp + 1e-8)
    return {"AUC": auc, "F1": f1, "ACC": acc, "SEN": sen, "SPE": spe}


# --- Train (2 epochs smoke test) ---
EPOCHS = 2
best_auc = -1.0
os.makedirs("/content/runs", exist_ok=True)
ckpt_path = "/content/runs/best.pt"
for epoch in range(1, EPOCHS + 1):
    model.train()
    running = 0.0
    n = 0
    t0 = time.time()
    for x, y in dl_train:
        x = x.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE)

        optimizer.zero_grad(set_to_none=True)
        with autocast(device_type="cuda", enabled=(DEVICE == "cuda")):
            logits = model(x)
            loss = criterion(logits, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running += loss.item() * x.size(0)
        n += x.size(0)

    scheduler.step()
    train_loss = running / max(1, n)
    metrics_val = evaluate(model, dl_val)
    print(
        f"[Epoch {epoch}/{EPOCHS}] loss={train_loss:.4f} | "
        f"val: AUC={metrics_val['AUC']:.3f}, F1={metrics_val['F1']:.3f}, "
        f"ACC={metrics_val['ACC']:.3f}, SEN={metrics_val['SEN']:.3f}, SPE={metrics_val['SPE']:.3f} "
        f"| time={time.time()-t0:.1f}s"
    )

    if metrics_val["AUC"] > best_auc:
        best_auc = metrics_val["AUC"]
        torch.save({"model": model.state_dict(), "metrics_val": metrics_val}, ckpt_path)
        print("  ✅ Saved new best:", ckpt_path)

print("\nDone. Best AUC on val:", best_auc, "| checkpoint:", ckpt_path)

Train size=1599 | Val size=1309 | Test size=1373 | pos_weight≈1.001
[Epoch 1/2] loss=0.4850 | val: AUC=0.856, F1=0.788, ACC=0.756, SEN=0.783, SPE=0.718 | time=382.0s
  ✅ Saved new best: /content/runs/best.pt
[Epoch 2/2] loss=0.3084 | val: AUC=0.955, F1=0.912, ACC=0.899, SEN=0.896, SPE=0.904 | time=381.7s
  ✅ Saved new best: /content/runs/best.pt

Done. Best AUC on val: 0.9549766439094501 | checkpoint: /content/runs/best.pt


In [ ]:
import numpy as np

# Evaluate on TEST set using best checkpoint
import torch
from dataset import MammogramFolders
from model import build_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader
from torchvision import transforms

DATA_ROOT = "/content/data"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
CKPT = "/content/runs/best.pt"

tfm_eval = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ]
)

ds_test = MammogramFolders(f"{DATA_ROOT}/test", transform=tfm_eval)
dl_test = DataLoader(
    ds_test, batch_size=32, shuffle=False, num_workers=2, pin_memory=True
)

model = build_model(pretrained=False).to(DEVICE)
state = torch.load(CKPT, map_location=DEVICE, weights_only=False)  # ⬅️ اصلاح اصلی
model.load_state_dict(state["model"])
model.eval()

ys, ps = [], []
with torch.no_grad():
    for x, y in dl_test:
        x = x.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE)
        logits = model(x)
        prob = torch.sigmoid(logits)
        ys.append(y.float().cpu().numpy())
        ps.append(prob.float().cpu().numpy())

y_true = np.concatenate(ys)
y_prob = np.concatenate(ps)
y_pred = (y_prob >= 0.5).astype(np.float32)

auc = roc_auc_score(y_true, y_prob)
f1 = f1_score(y_true, y_pred)
acc = accuracy_score(y_true, y_pred)
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
sen = tp / (tp + fn + 1e-8)
spe = tn / (tn + fp + 1e-8)

print(
    f"TEST -> AUC={auc:.3f} | F1={f1:.3f} | ACC={acc:.3f} | SEN={sen:.3f} | SPE={spe:.3f}"
)
print(f"Confusion Matrix counts: TN={tn}, FP={fp}, FN={fn}, TP={tp}")

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a91a0504fe0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a91a0504fe0>

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers

    if w.is_alive():
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
     if w.is_alive():  
        ^ ^  ^^^^^^^^^^^^^^^^^^^^^
^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
        assert self.

TEST -> AUC=0.946 | F1=0.891 | ACC=0.878 | SEN=0.861 | SPE=0.903
Confusion Matrix counts: TN=520, FP=56, FN=111, TP=686


In [ ]:
import glob

import os

import numpy as np
import torch
import torch.nn.functional as F
from dataset import MammogramFolders
from model import build_model
from PIL import Image
from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader
from torchvision import transforms

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
CKPT = "/content/runs/best.pt"
SAVE_DIR = "/content/gradcam"
os.makedirs(SAVE_DIR, exist_ok=True)

# 1) data + model
tfm = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ]
)
test_ds = MammogramFolders("/content/data/test", transform=tfm)
test_dl = DataLoader(
    test_ds, batch_size=16, shuffle=False, num_workers=0
)  # num_workers=0 برای جلوگیری از هشدارها

model = build_model(pretrained=False).to(DEVICE)
state = torch.load(CKPT, map_location=DEVICE, weights_only=False)
model.load_state_dict(state["model"])
model.eval()

# 2) پیش‌بینی همه‌ی تست برای پیدا کردن TP/FN
all_probs, all_trues = [], []
with torch.no_grad():
    for x, y in test_dl:
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        p = torch.sigmoid(model(x))
        all_probs.append(p.detach().cpu())
        all_trues.append(y.detach().cpu())
all_prob = torch.cat(all_probs).numpy()
all_true = torch.cat(all_trues).numpy().astype(int)
all_pred = (all_prob >= 0.5).astype(int).ravel()

# اندیس‌های TP و FN
tn, fp, fn, tp = confusion_matrix(all_true, all_pred).ravel()
tp_idx = np.where((all_true == 1) & (all_pred == 1))[0][:3]
fn_idx = np.where((all_true == 1) & (all_pred == 0))[0][:3]
sel_idx = np.concatenate([tp_idx, fn_idx])

# 3) پیاده‌سازی Grad-CAM روی لایه‌ی آخر ResNet18 (layer4)
# پیدا کردن feature map از layer4
target_module = model.features[-1]  # layer4
feats = None
grads = None


def fwd_hook(m, inp, out):  # save feature maps
    global feats
    feats = out.detach()


def bwd_hook(m, gin, gout):  # save grads
    global grads
    grads = gout[0].detach()


h1 = target_module.register_forward_hook(fwd_hook)
h2 = target_module.register_full_backward_hook(bwd_hook)

to_img = transforms.ToPILImage()

# برای دسترسی مستقیم به فایل‌ها (نام‌ها)
test_paths = test_ds.samples


def gradcam_for_index(i):
    path, label = test_paths[i]
    img0 = Image.open(path).convert("L").resize((512, 512))
    x = (
        transforms.functional.to_tensor(img0).repeat(3, 1, 1).unsqueeze(0).to(DEVICE)
    )  # 3ch
    x.requires_grad_(True)

    logits = model(x)  # [1]
    score = torch.sigmoid(logits)[0]  # احتمالِ mass
    model.zero_grad(set_to_none=True)
    score.backward()

    # CAM: وزن‌ها = میانگین فضاییِ گرادیان‌ها در هر کانال
    g = grads[0]  # [C,H,W]
    w = g.mean(dim=(1, 2), keepdim=True)  # [C,1,1]
    cam = (feats[0] * w).sum(0)  # [H,W]
    cam = F.relu(cam)
    cam = cam / (cam.max() + 1e-8)
    cam_np = cam.detach().cpu().numpy()

    # ترکیب heatmap با تصویر خاکستری
    import matplotlib.pyplot as plt

    plt.figure(figsize=(5, 5))
    plt.imshow(img0, cmap="gray")
    plt.imshow(
        cam_np, alpha=0.35
    )  # بدون انتخاب رنگ تا قانون رعایت شود (matplotlib default)
    title = f"prob={float(score):.2f} | true={int(label)} | pred={int(score>=0.5)}"
    plt.title(title)
    out_path = os.path.join(SAVE_DIR, f"gradcam_{i:05d}.png")
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(out_path, dpi=160)
    plt.close()
    return out_path, title, path


saved = []
for i in sel_idx:
    out_path, title, src = gradcam_for_index(int(i))
    saved.append((out_path, title, src))

h1.remove()
h2.remove()

print("Saved files:")
for p, t, s in saved:
    print(" -", p, " :: ", t, " :: src=", s)
print("\nTotal saved:", len(saved), "| Folder:", SAVE_DIR)

/tmp/ipython-input-2662472410.py:87: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  title = f"prob={float(score):.2f} | true={int(label)} | pred={int(score>=0.5)}"


Saved files:
 - /content/gradcam/gradcam_00576.png  ::  prob=1.00 | true=1 | pred=1  :: src= /content/data/test/mass/csv_p_01481__1-202.jpg
 - /content/gradcam/gradcam_00577.png  ::  prob=0.94 | true=1 | pred=1  :: src= /content/data/test/mass/csv_p_00915__1-289.jpg
 - /content/gradcam/gradcam_00578.png  ::  prob=0.78 | true=1 | pred=1  :: src= /content/data/test/mass/csv_p_00728__1-008.jpg
 - /content/gradcam/gradcam_00583.png  ::  prob=0.04 | true=1 | pred=0  :: src= /content/data/test/mass/csv_p_01697__2-164.jpg
 - /content/gradcam/gradcam_00591.png  ::  prob=0.22 | true=1 | pred=0  :: src= /content/data/test/mass/csv_p_00092__2-273.jpg
 - /content/gradcam/gradcam_00601.png  ::  prob=0.33 | true=1 | pred=0  :: src= /content/data/test/mass/csv_p_00881__1-219.jpg

Total saved: 6 | Folder: /content/gradcam


In [ ]:
# Create ROC curve + Confusion Matrix figures for TEST split
import os

import matplotlib.pyplot as plt
import numpy as np
import torch
from dataset import MammogramFolders
from model import build_model
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from torch.utils.data import DataLoader
from torchvision import transforms

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
CKPT = "/content/runs/best.pt"

tfm = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ]
)

ds_test = MammogramFolders("/content/data/test", transform=tfm)
dl_test = DataLoader(ds_test, batch_size=64, shuffle=False, num_workers=0)

model = build_model(pretrained=False).to(DEVICE)
state = torch.load(CKPT, map_location=DEVICE, weights_only=False)
model.load_state_dict(state["model"])
model.eval()

ys, ps = [], []
with torch.no_grad():
    for x, y in dl_test:
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        p = torch.sigmoid(model(x))
        ys.append(y.cpu().numpy())
        ps.append(p.cpu().numpy())
y_true = np.concatenate(ys).astype(int)
y_prob = np.concatenate(ps)

# ROC
fpr, tpr, thr = roc_curve(y_true, y_prob)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC – Test")
plt.legend(loc="lower right")
plt.tight_layout()
plt.savefig("/content/fig_roc_test.png", dpi=160)
plt.close()

# Confusion Matrix (threshold=0.5)
y_pred = (y_prob >= 0.5).astype(int)
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(5, 5))
plt.imshow(cm, interpolation="nearest")
for i in range(2):
    for j in range(2):
        plt.text(j, i, cm[i, j], ha="center", va="center")
plt.title("Confusion Matrix – Test")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.savefig("/content/fig_cm_test.png", dpi=160)
plt.close()

print("Saved: /content/fig_roc_test.png")
print("Saved: /content/fig_cm_test.png")

Saved: /content/fig_roc_test.png
Saved: /content/fig_cm_test.png


In [ ]:
%%writefile /content/model.py
import torch
import torch.nn as nn
from torchvision.models import resnet18

class LiteAttention(nn.Module):
    def __init__(self, c, r=8):
        super().__init__()
        r = max(1, c // r)
        # Channel attention (SE)
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(c, r, kernel_size=1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(r, c, kernel_size=1, bias=False),
            nn.Sigmoid()
        )
        # Spatial attention
        self.spatial = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=7, padding=3, bias=False),
            nn.Sigmoid()
        )
    def forward(self, x):
        # channel
        w_c = self.se(x); x = x * w_c
        # spatial
        avg_map = torch.mean(x, dim=1, keepdim=True)
        max_map, _ = torch.max(x, dim=1, keepdim=True)
        w_s = self.spatial(torch.cat([avg_map, max_map], dim=1))
        return x * w_s

class ResNet18LiteAtt(nn.Module):
    def __init__(self, pretrained=True, use_attn=True):
        super().__init__()
        self.backbone = resnet18(weights="IMAGENET1K_V1" if pretrained else None)
        self.features = nn.Sequential(
            self.backbone.conv1, self.backbone.bn1, self.backbone.relu, self.backbone.maxpool,
            self.backbone.layer1, self.backbone.layer2, self.backbone.layer3, self.backbone.layer4
        )
        c = 512
        self.use_attn = use_attn
        self.attn = LiteAttention(c, r=8) if use_attn else nn.Identity()
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(c, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )
    def forward(self, x):
        f = self.features(x)
        f = self.attn(f)
        out = self.head(f)
        return out.squeeze(1)

def build_model(pretrained=True, use_attn=True):
    return ResNet18LiteAtt(pretrained=pretrained, use_attn=use_attn)


Overwriting /content/model.py


In [ ]:
import importlib
import sys

if "model" in sys.modules:
    importlib.reload(sys.modules["model"])
else:
    import model  # noqa

from model import build_model

print("build_model signature ready.")

build_model signature ready.


In [ ]:
#  B2: Train Baseline (ResNet18 بدون Attention) و ارزیابی
import os
import random
import time

import numpy as np
import torch
from dataset import MammogramFolders
from model import build_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from torch.amp import GradScaler
from torch.amp import autocast
from torch.utils.data import DataLoader
from torchvision import transforms

# reproducibility سبک
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

DATA_ROOT = "/content/data"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# transforms
tfm_train = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomRotation(5),
        transforms.ToTensor(),
    ]
)
tfm_eval = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ]
)

# datasets/loaders
ds_train = MammogramFolders(f"{DATA_ROOT}/train", transform=tfm_train)
ds_val = MammogramFolders(f"{DATA_ROOT}/val", transform=tfm_eval)
ds_test = MammogramFolders(f"{DATA_ROOT}/test", transform=tfm_eval)

dl_train = DataLoader(
    ds_train, batch_size=16, shuffle=True, num_workers=2, pin_memory=True
)
dl_val = DataLoader(
    ds_val, batch_size=32, shuffle=False, num_workers=2, pin_memory=True
)
dl_test = DataLoader(
    ds_test, batch_size=32, shuffle=False, num_workers=2, pin_memory=True
)

# model/loss/opt
model = build_model(pretrained=True, use_attn=False).to(DEVICE)  # ⬅️ بدون attention
pos_weight = torch.tensor([ds_train.pos_weight()], device=DEVICE)
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)
scaler = GradScaler(device=DEVICE if DEVICE == "cuda" else "cpu")


def eval_metrics(model, loader):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(DEVICE, non_blocking=True)
            y = y.to(DEVICE)
            p = torch.sigmoid(model(x))
            ys.append(y.float().cpu().numpy())
            ps.append(p.float().cpu().numpy())
    import numpy as np

    y_true = np.concatenate(ys)
    y_prob = np.concatenate(ps)
    y_pred = (y_prob >= 0.5).astype(np.float32)
    auc = roc_auc_score(y_true, y_prob)
    f1 = f1_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sen = tp / (tp + fn + 1e-8)
    spe = tn / (tn + fp + 1e-8)
    return {"AUC": auc, "F1": f1, "ACC": acc, "SEN": sen, "SPE": spe}


# train 5 epochs (quick ablation)
EPOCHS = 5
best_auc = -1
os.makedirs("/content/runs", exist_ok=True)
ckpt_noattn = "/content/runs/baseline_noattn.pt"

for ep in range(1, EPOCHS + 1):
    model.train()
    t0 = time.time()
    run = 0
    n = 0
    for x, y in dl_train:
        x = x.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE)
        optimizer.zero_grad(set_to_none=True)
        with autocast(device_type="cuda", enabled=(DEVICE == "cuda")):
            logits = model(x)
            loss = criterion(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        run += loss.item() * x.size(0)
        n += x.size(0)
    scheduler.step()
    tr_loss = run / max(1, n)
    mv = eval_metrics(model, dl_val)
    print(
        f"[NoAttn Ep{ep}/{EPOCHS}] loss={tr_loss:.4f} | "
        f"val: AUC={mv['AUC']:.3f}, F1={mv['F1']:.3f}, ACC={mv['ACC']:.3f}, "
        f"SEN={mv['SEN']:.3f}, SPE={mv['SPE']:.3f} | {time.time()-t0:.1f}s"
    )
    if mv["AUC"] > best_auc:
        best_auc = mv["AUC"]
        torch.save({"model": model.state_dict(), "metrics_val": mv}, ckpt_noattn)
        print("  ✅ Saved best baseline:", ckpt_noattn)

# test metrics for baseline
state = torch.load(ckpt_noattn, map_location=DEVICE, weights_only=False)
model.load_state_dict(state["model"])
model.eval()
mb = eval_metrics(model, dl_test)
print(
    f"\nBASELINE (NoAttn) TEST -> AUC={mb['AUC']:.3f} | F1={mb['F1']:.3f} | "
    f"ACC={mb['ACC']:.3f} | SEN={mb['SEN']:.3f} | SPE={mb['SPE']:.3f}"
)
print("Checkpoint:", ckpt_noattn)

[NoAttn Ep1/5] loss=0.4793 | val: AUC=0.909, F1=0.866, ACC=0.834, SEN=0.922, SPE=0.713 | 424.0s
  ✅ Saved best baseline: /content/runs/baseline_noattn.pt
[NoAttn Ep2/5] loss=0.3163 | val: AUC=0.949, F1=0.904, ACC=0.884, SEN=0.939, SPE=0.807 | 393.5s
  ✅ Saved best baseline: /content/runs/baseline_noattn.pt
[NoAttn Ep3/5] loss=0.2575 | val: AUC=0.961, F1=0.919, ACC=0.905, SEN=0.928, SPE=0.875 | 399.1s
  ✅ Saved best baseline: /content/runs/baseline_noattn.pt
[NoAttn Ep4/5] loss=0.2148 | val: AUC=0.966, F1=0.932, ACC=0.923, SEN=0.908, SPE=0.944 | 384.7s
  ✅ Saved best baseline: /content/runs/baseline_noattn.pt
[NoAttn Ep5/5] loss=0.1849 | val: AUC=0.971, F1=0.938, ACC=0.929, SEN=0.920, SPE=0.942 | 393.9s
  ✅ Saved best baseline: /content/runs/baseline_noattn.pt

BASELINE (NoAttn) TEST -> AUC=0.965 | F1=0.922 | ACC=0.912 | SEN=0.896 | SPE=0.934
Checkpoint: /content/runs/baseline_noattn.pt


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# B3: Compare current checkpoints (unequal epochs)
import torch
from dataset import MammogramFolders
from model import build_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader
from torchvision import transforms

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATA_ROOT = "/content/data"
CKPT_ATTN = "/content/runs/best.pt"  # (+Attention, 2 epochs)
CKPT_NOAT = "/content/runs/baseline_noattn.pt"  # (NoAttention, 5 epochs)

tfm = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ]
)

ds_test = MammogramFolders(f"{DATA_ROOT}/test", transform=tfm)
dl_test = DataLoader(
    ds_test, batch_size=64, shuffle=False, num_workers=0, pin_memory=True
)


def eval_ckpt(ckpt_path, use_attn):
    m = build_model(pretrained=False, use_attn=use_attn).to(DEVICE)
    st = torch.load(ckpt_path, map_location=DEVICE, weights_only=False)
    m.load_state_dict(st["model"])
    m.eval()
    ys, ps = [], []
    with torch.no_grad():
        for x, y in dl_test:
            x = x.to(DEVICE)
            y = y.to(DEVICE)
            p = torch.sigmoid(m(x))
            ys.append(y.float().cpu().numpy())
            ps.append(p.float().cpu().numpy())
    y_true = np.concatenate(ys)
    y_prob = np.concatenate(ps)
    y_pred = (y_prob >= 0.5).astype(np.float32)
    auc = roc_auc_score(y_true, y_prob)
    from sklearn.metrics import f1_score

    f1 = f1_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sen = tp / (tp + fn + 1e-8)
    spe = tn / (tn + fp + 1e-8)
    return {"AUC": auc, "F1": f1, "ACC": acc, "SEN": sen, "SPE": spe}


m_attn = eval_ckpt(CKPT_ATTN, use_attn=True)
m_noat = eval_ckpt(CKPT_NOAT, use_attn=False)

print("(+Attention, 2ep) TEST:", m_attn)
print("(NoAttention, 5ep) TEST:", m_noat)

# بارچارت جمع‌وجور برای اسلاید
metrics = ["AUC", "F1", "ACC", "SEN", "SPE"]
attn_vals = [m_attn[k] for k in metrics]
noat_vals = [m_noat[k] for k in metrics]

plt.figure(figsize=(6.5, 3.0))
x = np.arange(len(metrics))
w = 0.35
plt.bar(x - w / 2, attn_vals, width=w, label="+Attention (2ep)")
plt.bar(x + w / 2, noat_vals, width=w, label="Baseline (5ep)")
plt.xticks(x, metrics)
plt.ylim(0.6, 1.0)
plt.ylabel("Score")
plt.title("Comparison on TEST (⚠️ different epochs)")
plt.legend()
plt.tight_layout()
plt.savefig("/content/compare_now.png", dpi=180)
plt.close()
print("Saved figure:", "/content/compare_now.png")

(+Attention, 2ep) TEST: {'AUC': np.float64(0.9462263000139411), 'F1': 0.8914879792072774, 'ACC': 0.8783685360524399, 'SEN': np.float64(0.8607277289728892), 'SPE': np.float64(0.9027777777621045)}
(NoAttention, 5ep) TEST: {'AUC': np.float64(0.9651601491705004), 'F1': 0.921885087153002, 'ACC': 0.9118718135469774, 'SEN': np.float64(0.8958594730125989), 'SPE': np.float64(0.934027777761562)}
Saved figure: /content/compare_now.png


In [ ]:

from google.colab import drive

drive.mount("/content/drive")

import os
import sys

BASE_DIR = "/content/drive/MyDrive/mammo_cad"
os.makedirs(BASE_DIR, exist_ok=True)

# مسیرها را برای import اضافه کن
for p in ["/content", BASE_DIR]:
    if p not in sys.path:
        sys.path.insert(0, p)

# اگر فایل‌ها نیستند، بسازیم‌شان (آخرین نسخه با use_attn)
if not os.path.exists("/content/dataset.py"):
    with open("/content/dataset.py", "w") as f:
        f.write(
            r"""
import os, glob
from PIL import Image
from torch.utils.data import Dataset
import torch

class MammogramFolders(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.samples = []
        for label_name in ["normal", "mass"]:
            y = 0 if label_name=="normal" else 1
            for p in glob.glob(os.path.join(root, label_name, "*")):
                if p.lower().endswith((".png",".jpg",".jpeg",".bmp",".tif",".tiff")):
                    self.samples.append((p, y))
        if not self.samples:
            raise RuntimeError(f"No images under {root}")
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        p, y = self.samples[idx]
        img = Image.open(p).convert("L")
        if self.transform: img = self.transform(img)
        return img, torch.tensor(y, dtype=torch.float32)
    def pos_weight(self):
        n_pos = sum(1 for _,y in self.samples if y==1)
        n_neg = len(self.samples)-n_pos
        return (n_neg / max(1,n_pos)) if n_pos>0 else 1.0
"""
        )
    print("✅ wrote /content/dataset.py")
else:
    print("ℹ️ dataset.py already exists at /content")

if not os.path.exists("/content/model.py"):
    with open("/content/model.py", "w") as f:
        f.write(
            r"""
import torch
import torch.nn as nn
from torchvision.models import resnet18

class LiteAttention(nn.Module):
    def __init__(self, c, r=8):
        super().__init__()
        r = max(1, c//r)
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(c, r, 1, bias=False), nn.ReLU(inplace=True),
            nn.Conv2d(r, c, 1, bias=False), nn.Sigmoid()
        )
        self.spatial = nn.Sequential(
            nn.Conv2d(2, 1, 7, padding=3, bias=False), nn.Sigmoid()
        )
    def forward(self, x):
        x = x * self.se(x)
        avg = torch.mean(x, 1, keepdim=True)
        mx, _ = torch.max(x, 1, keepdim=True)
        attn = self.spatial(torch.cat([avg, mx], 1))
        return x * attn

class ResNet18LiteAtt(nn.Module):
    def __init__(self, pretrained=True, use_attn=True):
        super().__init__()
        self.backbone = resnet18(weights="IMAGENET1K_V1" if pretrained else None)
        self.features = nn.Sequential(
            self.backbone.conv1, self.backbone.bn1, self.backbone.relu, self.backbone.maxpool,
            self.backbone.layer1, self.backbone.layer2, self.backbone.layer3, self.backbone.layer4
        )
        c = 512
        self.attn = LiteAttention(c) if use_attn else nn.Identity()
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), nn.Flatten(),
            nn.Linear(c,128), nn.ReLU(inplace=True), nn.Dropout(0.2),
            nn.Linear(128,1)
        )
    def forward(self, x):
        f = self.features(x); f = self.attn(f); out = self.head(f)
        return out.squeeze(1)

def build_model(pretrained=True, use_attn=True):
    return ResNet18LiteAtt(pretrained=pretrained, use_attn=use_attn)
"""
        )
    print("✅ wrote /content/model.py")
else:
    print("ℹ️ model.py already exists at /content")

# تست import
try:
    import importlib

    import dataset
    import model

    importlib.reload(dataset)
    importlib.reload(model)
    from dataset import MammogramFolders
    from model import build_model

    print("✅ Import OK: MammogramFolders, build_model are available.")
except Exception as e:
    print("❌ Import failed:", e)

import shlex

# نمایش کجا هستیم و چه فایل‌هایی داریم
import subprocess

print("\nWorking dir:", os.getcwd())
print("Listing /content:")
print(
    subprocess.run(
        shlex.split("bash -lc 'ls -lah /content | sed -n \"1,80p\"'"),
        capture_output=True,
        text=True,
    ).stdout
)

Mounted at /content/drive
✅ wrote /content/dataset.py
✅ wrote /content/model.py
✅ Import OK: MammogramFolders, build_model are available.

Working dir: /content
Listing /content:
total 32K
drwxr-xr-x 1 root root 4.0K Oct 19 16:45 .
drwxr-xr-x 1 root root 4.0K Oct 19 16:00 ..
drwxr-xr-x 4 root root 4.0K Oct 16 13:41 .config
-rw-r--r-- 1 root root 1.1K Oct 19 16:45 dataset.py
drwx------ 5 root root 4.0K Oct 19 16:45 drive
-rw-r--r-- 1 root root 1.7K Oct 19 16:45 model.py
drwxr-xr-x 2 root root 4.0K Oct 19 16:45 __pycache__
drwxr-xr-x 1 root root 4.0K Oct 16 13:41 sample_data



In [ ]:
import glob
import hashlib

# R2) Build or reuse patient-level split under Drive
import os
import random
import re
import shutil
from collections import defaultdict

import pandas as pd

BASE_DIR = "/content/drive/MyDrive/mammo_cad"
RAW_DIR = f"{BASE_DIR}/raw"
OUT_DIR = f"{BASE_DIR}/data_split"  # پایدار داخل Drive
CAP_PER_CLASS = 800
SEED = 42


def extract_uid(s):
    m = re.search(r"(1(?:\.\d+){3,})", str(s))
    return m.group(1) if m else None


def build_split(
    raw_dir=RAW_DIR, out_dir=OUT_DIR, cap_per_class=CAP_PER_CLASS, seed=SEED
):
    csv_dir = os.path.join(raw_dir, "csv")
    jpeg_dir = os.path.join(raw_dir, "jpeg")
    os.makedirs(out_dir, exist_ok=True)

    # 1) Mass studies (CSV)
    mass_csvs = glob.glob(os.path.join(csv_dir, "mass_case_description_*_set.csv"))
    mass_study_uids, mass_pid_by_study = set(), {}
    for c in mass_csvs:
        df = pd.read_csv(c)
        for _, row in df.iterrows():
            for col in ["image file path", "cropped image file path"]:
                u = extract_uid(row.get(col, ""))
                if u:
                    mass_study_uids.add(u)
                    pid = str(row.get("patient_id", "")).strip().lower()
                    if pid and u not in mass_pid_by_study:
                        mass_pid_by_study[u] = pid

    # 2) calc برای حذف
    calc_study = set()
    for c in glob.glob(os.path.join(csv_dir, "calc_case_description_*_set.csv")):
        df = pd.read_csv(c)
        for _, row in df.iterrows():
            u = extract_uid(row.get("image file path", ""))
            if u:
                calc_study.add(u)

    # 3) نگاشت Study↔Image از dicom_info.csv
    info = pd.read_csv(os.path.join(csv_dir, "dicom_info.csv"))
    study_col = info.get("StudyInstanceUID", "").apply(extract_uid)
    image_col = info.get("image_path", "").apply(extract_uid)
    study2images, image2study = defaultdict(set), {}
    for s_uid, i_uid in zip(study_col, image_col):
        if isinstance(s_uid, str) and isinstance(i_uid, str):
            study2images[s_uid].add(i_uid)
            image2study[i_uid] = s_uid

    mass_uids_jpeg = set().union(
        *[study2images[s] for s in mass_study_uids if s in study2images]
    )
    calc_uids_jpeg = set().union(
        *[study2images[s] for s in calc_study if s in study2images]
    )

    # 4) گردآوری نمونه‌ها به‌صورت patient-centric
    def pid_from_path(p):
        parent = os.path.dirname(p).lower()
        return "pid_" + hashlib.md5(parent.encode()).hexdigest()[:10]

    by_patient = defaultdict(list)
    imgs = glob.glob(os.path.join(jpeg_dir, "**/*.jpg"), recursive=True) + glob.glob(
        os.path.join(jpeg_dir, "**/*.jpeg"), recursive=True
    )
    for p in imgs:
        uid = extract_uid(p)
        if not uid or uid in calc_uids_jpeg:
            continue
        if uid in mass_uids_jpeg:
            lab = "mass"
            stu = image2study.get(uid, None)
            if stu and stu in mass_pid_by_study:
                pid = "csv_" + mass_pid_by_study[stu]
            else:
                pid = pid_from_path(p)
        else:
            lab = "normal"
            pid = pid_from_path(p)
        by_patient[pid].append((p, lab))

    # 5) split 70/15/15
    patients = list(by_patient.keys())
    random.seed(seed)
    random.shuffle(patients)
    n = len(patients)
    train = set(patients[: int(0.7 * n)])
    val = set(patients[int(0.7 * n) : int(0.85 * n)])
    test = set(patients[int(0.85 * n) :])

    # 6) ساخت دایرکتوری‌ها
    for sp in ["train", "val", "test"]:
        for c in ["mass", "normal"]:
            os.makedirs(os.path.join(out_dir, sp, c), exist_ok=True)
    counters = {
        ("train", "mass"): 0,
        ("train", "normal"): 0,
        ("val", "mass"): 0,
        ("val", "normal"): 0,
        ("test", "mass"): 0,
        ("test", "normal"): 0,
    }

    def copy_group(pids, split):
        for pid in pids:
            for src, lab in by_patient[pid]:
                key = (split, lab)
                if counters[key] >= cap_per_class:
                    continue
                dst = os.path.join(
                    out_dir, split, lab, f"{pid}__{os.path.basename(src)}"
                )
                if not os.path.exists(dst):
                    shutil.copy2(src, dst)
                counters[key] += 1

    copy_group(train, "train")
    copy_group(val, "val")
    copy_group(test, "test")

    # 7) گزارش
    def count(d):
        return len(glob.glob(os.path.join(d, "*")))

    summary = {
        "train": {
            "mass": count(os.path.join(out_dir, "train", "mass")),
            "normal": count(os.path.join(out_dir, "train", "normal")),
        },
        "val": {
            "mass": count(os.path.join(out_dir, "val", "mass")),
            "normal": count(os.path.join(out_dir, "val", "normal")),
        },
        "test": {
            "mass": count(os.path.join(out_dir, "test", "mass")),
            "normal": count(os.path.join(out_dir, "test", "normal")),
        },
    }
    return summary


# اگر قبلاً ساخته شده باشد، از همان استفاده می‌کنیم
already = (
    sum(
        len(glob.glob(os.path.join(OUT_DIR, sp, c, "*")))
        for sp in ["train", "val", "test"]
        for c in ["mass", "normal"]
    )
    > 0
)

if already:
    print("✅ data_split already exists in Drive:", OUT_DIR)
else:
    print("⏳ Building patient-level split under:", OUT_DIR)
    summary = build_split()
    print("✅ Built split. Summary:", summary)

# در هر صورت، یک لینک نمادین به /content/data برای کدهای بعدی می‌سازیم
import os

if os.path.islink("/content/data") or os.path.exists("/content/data"):
    pass
else:
    os.symlink(OUT_DIR, "/content/data")
print("DATA ready at:", "/content/data")
# شمارش نهایی
final_counts = {
    "train": {
        "mass": len(glob.glob("/content/data/train/mass/*")),
        "normal": len(glob.glob("/content/data/train/normal/*")),
    },
    "val": {
        "mass": len(glob.glob("/content/data/val/mass/*")),
        "normal": len(glob.glob("/content/data/val/normal/*")),
    },
    "test": {
        "mass": len(glob.glob("/content/data/test/mass/*")),
        "normal": len(glob.glob("/content/data/test/normal/*")),
    },
}
print("Final counts:", final_counts)

✅ data_split already exists in Drive: /content/drive/MyDrive/mammo_cad/data_split
DATA ready at: /content/data
Final counts: {'train': {'mass': 798, 'normal': 800}, 'val': {'mass': 265, 'normal': 241}, 'test': {'mass': 0, 'normal': 0}}


In [ ]:
import glob
import hashlib

# R3) Force rebuild patient-level split so that TEST is non-empty
import os
import random
import re
import shutil
from collections import defaultdict

import pandas as pd

BASE_DIR = "/content/drive/MyDrive/mammo_cad"
RAW_DIR = f"{BASE_DIR}/raw"
OUT_DIR = f"{BASE_DIR}/data_split"
CAP_PER_CLASS = 800
SEED = 42


def extract_uid(s):
    m = re.search(r"(1(?:\.\d+){3,})", str(s))
    return m.group(1) if m else None


def rebuild_split(
    raw_dir=RAW_DIR, out_dir=OUT_DIR, cap_per_class=CAP_PER_CLASS, seed=SEED
):
    # clean old split safely
    for sp in ["train", "val", "test"]:
        for c in ["mass", "normal"]:
            d = os.path.join(out_dir, sp, c)
            if os.path.isdir(d):
                for f in glob.glob(os.path.join(d, "*")):
                    try:
                        os.remove(f)
                    except:
                        pass
            else:
                os.makedirs(d, exist_ok=True)

    csv_dir = os.path.join(raw_dir, "csv")
    jpeg_dir = os.path.join(raw_dir, "jpeg")

    # 1) Mass studies (CSV)
    mass_csvs = glob.glob(os.path.join(csv_dir, "mass_case_description_*_set.csv"))
    mass_study_uids, mass_pid_by_study = set(), {}
    for c in mass_csvs:
        df = pd.read_csv(c)
        for _, row in df.iterrows():
            for col in ["image file path", "cropped image file path"]:
                u = extract_uid(row.get(col, ""))
                if u:
                    mass_study_uids.add(u)
                    pid = str(row.get("patient_id", "")).strip().lower()
                    if pid and u not in mass_pid_by_study:
                        mass_pid_by_study[u] = pid

    # 2) calc برای حذف
    calc_study = set()
    for c in glob.glob(os.path.join(csv_dir, "calc_case_description_*_set.csv")):
        df = pd.read_csv(c)
        for _, row in df.iterrows():
            u = extract_uid(row.get("image file path", ""))
            if u:
                calc_study.add(u)

    # 3) Study↔Image map
    info = pd.read_csv(os.path.join(csv_dir, "dicom_info.csv"))
    study_col = info.get("StudyInstanceUID", "").apply(extract_uid)
    image_col = info.get("image_path", "").apply(extract_uid)
    from collections import defaultdict

    study2images, image2study = defaultdict(set), {}
    for s_uid, i_uid in zip(study_col, image_col):
        if isinstance(s_uid, str) and isinstance(i_uid, str):
            study2images[s_uid].add(i_uid)
            image2study[i_uid] = s_uid

    mass_uids_jpeg = set().union(
        *[study2images[s] for s in mass_study_uids if s in study2images]
    )
    calc_uids_jpeg = set().union(
        *[study2images[s] for s in calc_study if s in study2images]
    )

    # 4) patient-centric grouping
    def pid_from_path(p):
        parent = os.path.dirname(p).lower()
        return "pid_" + hashlib.md5(parent.encode()).hexdigest()[:10]

    by_patient = defaultdict(list)
    imgs = glob.glob(os.path.join(jpeg_dir, "**/*.jpg"), recursive=True) + glob.glob(
        os.path.join(jpeg_dir, "**/*.jpeg"), recursive=True
    )
    for p in imgs:
        uid = extract_uid(p)
        if not uid or uid in calc_uids_jpeg:
            continue
        if uid in mass_uids_jpeg:
            lab = "mass"
            stu = image2study.get(uid, None)
            if stu and stu in mass_pid_by_study:
                pid = "csv_" + mass_pid_by_study[stu]
            else:
                pid = pid_from_path(p)
        else:
            lab = "normal"
            pid = pid_from_path(p)
        by_patient[pid].append((p, lab))

    # 5) 70/15/15 split
    patients = list(by_patient.keys())
    random.seed(seed)
    random.shuffle(patients)
    n = len(patients)
    train = set(patients[: int(0.7 * n)])
    val = set(patients[int(0.7 * n) : int(0.85 * n)])
    test = set(patients[int(0.85 * n) :])

    # 6) copy with caps per class and per split
    caps = {
        ("train", "mass"): cap_per_class,
        ("train", "normal"): cap_per_class,
        ("val", "mass"): max(200, cap_per_class // 2),
        ("val", "normal"): max(200, cap_per_class // 2),
        ("test", "mass"): max(200, cap_per_class // 2),
        ("test", "normal"): max(200, cap_per_class // 2),
    }
    counters = {k: 0 for k in caps}

    def copy_group(pids, split):
        for pid in pids:
            for src, lab in by_patient[pid]:
                key = (split, lab)
                if counters[key] >= caps[key]:
                    continue
                dst = os.path.join(
                    out_dir, split, lab, f"{pid}__{os.path.basename(src)}"
                )
                if not os.path.exists(dst):
                    try:
                        shutil.copy2(src, dst)
                        counters[key] += 1
                    except:
                        pass

    copy_group(train, "train")
    copy_group(val, "val")
    copy_group(test, "test")

    def count(d):
        return len(glob.glob(os.path.join(d, "*")))

    summary = {
        "train": {
            "mass": count(os.path.join(out_dir, "train", "mass")),
            "normal": count(os.path.join(out_dir, "train", "normal")),
        },
        "val": {
            "mass": count(os.path.join(out_dir, "val", "mass")),
            "normal": count(os.path.join(out_dir, "val", "normal")),
        },
        "test": {
            "mass": count(os.path.join(out_dir, "test", "mass")),
            "normal": count(os.path.join(out_dir, "test", "normal")),
        },
    }
    return summary


summary = rebuild_split()
print("✅ Rebuilt split. Summary:", summary)

# link for /content/data
if not (os.path.islink("/content/data") or os.path.exists("/content/data")):
    os.symlink(OUT_DIR, "/content/data")
print("DATA ready at:", "/content/data")

# final counts print again
import glob

final_counts = {
    "train": {
        "mass": len(glob.glob("/content/data/train/mass/*")),
        "normal": len(glob.glob("/content/data/train/normal/*")),
    },
    "val": {
        "mass": len(glob.glob("/content/data/val/mass/*")),
        "normal": len(glob.glob("/content/data/val/normal/*")),
    },
    "test": {
        "mass": len(glob.glob("/content/data/test/mass/*")),
        "normal": len(glob.glob("/content/data/test/normal/*")),
    },
}
print("Final counts:", final_counts)

✅ Rebuilt split. Summary: {'train': {'mass': 800, 'normal': 800}, 'val': {'mass': 400, 'normal': 400}, 'test': {'mass': 400, 'normal': 400}}
DATA ready at: /content/data
Final counts: {'train': {'mass': 800, 'normal': 800}, 'val': {'mass': 400, 'normal': 400}, 'test': {'mass': 400, 'normal': 400}}


In [ ]:
# B4) Train +Attention for 5 epochs (same settings as baseline) and evaluate on TEST
import os
import random
import time

import numpy as np
import torch
from dataset import MammogramFolders
from model import build_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from torch.amp import GradScaler
from torch.amp import autocast
from torch.utils.data import DataLoader
from torchvision import transforms

# reproducibility (سبک)
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

DATA_ROOT = "/content/data"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

tfm_train = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomRotation(5),
        transforms.ToTensor(),
    ]
)
tfm_eval = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ]
)

ds_train = MammogramFolders(f"{DATA_ROOT}/train", transform=tfm_train)
ds_val = MammogramFolders(f"{DATA_ROOT}/val", transform=tfm_eval)
ds_test = MammogramFolders(f"{DATA_ROOT}/test", transform=tfm_eval)

dl_train = DataLoader(
    ds_train, batch_size=16, shuffle=True, num_workers=2, pin_memory=True
)
dl_val = DataLoader(
    ds_val, batch_size=32, shuffle=False, num_workers=2, pin_memory=True
)
dl_test = DataLoader(
    ds_test, batch_size=32, shuffle=False, num_workers=2, pin_memory=True
)

model = build_model(pretrained=True, use_attn=True).to(DEVICE)
pos_weight = torch.tensor([ds_train.pos_weight()], device=DEVICE)
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)
scaler = GradScaler(device=DEVICE if DEVICE == "cuda" else "cpu")


def eval_metrics(model, loader):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(DEVICE, non_blocking=True)
            y = y.to(DEVICE)
            p = torch.sigmoid(model(x))
            ys.append(y.float().cpu().numpy())
            ps.append(p.float().cpu().numpy())
    y_true = np.concatenate(ys)
    y_prob = np.concatenate(ps)
    y_pred = (y_prob >= 0.5).astype(np.float32)
    auc = roc_auc_score(y_true, y_prob)
    f1 = f1_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sen = tp / (tp + fn + 1e-8)
    spe = tn / (tn + fp + 1e-8)
    return {"AUC": auc, "F1": f1, "ACC": acc, "SEN": sen, "SPE": spe}


EPOCHS = 5
best_auc = -1
os.makedirs("/content/runs", exist_ok=True)
ckpt_attn5 = "/content/runs/attn_5ep.pt"

for ep in range(1, EPOCHS + 1):
    model.train()
    t0 = time.time()
    run = 0
    n = 0
    for x, y in dl_train:
        x = x.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE)
        optimizer.zero_grad(set_to_none=True)
        with autocast(device_type="cuda", enabled=(DEVICE == "cuda")):
            logits = model(x)
            loss = criterion(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        run += loss.item() * x.size(0)
        n += x.size(0)
    scheduler.step()
    tr_loss = run / max(1, n)
    mv = eval_metrics(model, dl_val)
    print(
        f"[+Attn Ep{ep}/{EPOCHS}] loss={tr_loss:.4f} | "
        f"val: AUC={mv['AUC']:.3f}, F1={mv['F1']:.3f}, ACC={mv['ACC']:.3f}, "
        f"SEN={mv['SEN']:.3f}, SPE={mv['SPE']:.3f} | {time.time()-t0:.1f}s"
    )
    if mv["AUC"] > best_auc:
        best_auc = mv["AUC"]
        torch.save({"model": model.state_dict(), "metrics_val": mv}, ckpt_attn5)
        print("  ✅ Saved best +Attention:", ckpt_attn5)

# TEST با بهترین ایپاک
state = torch.load(ckpt_attn5, map_location=DEVICE, weights_only=False)
model.load_state_dict(state["model"])
model.eval()
mb = eval_metrics(model, dl_test)
print(
    f"\n+ATTENTION (5ep) TEST -> AUC={mb['AUC']:.3f} | F1={mb['F1']:.3f} | "
    f"ACC={mb['ACC']:.3f} | SEN={mb['SEN']:.3f} | SPE={mb['SPE']:.3f}"
)
print("Checkpoint:", ckpt_attn5)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 192MB/s]


[+Attn Ep1/5] loss=0.5073 | val: AUC=0.886, F1=0.773, ACC=0.792, SEN=0.707, SPE=0.877 | 158.0s
  ✅ Saved best +Attention: /content/runs/attn_5ep.pt
[+Attn Ep2/5] loss=0.3114 | val: AUC=0.949, F1=0.865, ACC=0.870, SEN=0.835, SPE=0.905 | 155.0s
  ✅ Saved best +Attention: /content/runs/attn_5ep.pt
[+Attn Ep3/5] loss=0.2520 | val: AUC=0.966, F1=0.919, ACC=0.917, SEN=0.932, SPE=0.902 | 156.5s
  ✅ Saved best +Attention: /content/runs/attn_5ep.pt
[+Attn Ep4/5] loss=0.2048 | val: AUC=0.971, F1=0.913, ACC=0.907, SEN=0.970, SPE=0.845 | 157.1s
  ✅ Saved best +Attention: /content/runs/attn_5ep.pt
[+Attn Ep5/5] loss=0.1911 | val: AUC=0.979, F1=0.944, ACC=0.944, SEN=0.955, SPE=0.932 | 156.9s
  ✅ Saved best +Attention: /content/runs/attn_5ep.pt

+ATTENTION (5ep) TEST -> AUC=0.973 | F1=0.925 | ACC=0.924 | SEN=0.937 | SPE=0.910
Checkpoint: /content/runs/attn_5ep.pt


In [ ]:
# Re-train BASELINE (No Attention) for 5 epochs so we can compare again
import os
import random
import time

import numpy as np
import torch
from dataset import MammogramFolders
from model import build_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from torch.amp import GradScaler
from torch.amp import autocast
from torch.utils.data import DataLoader
from torchvision import transforms

# reproducibility سبک
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

DATA_ROOT = "/content/data"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# transforms
tfm_train = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomRotation(5),
        transforms.ToTensor(),
    ]
)
tfm_eval = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ]
)


ds_train = MammogramFolders(f"{DATA_ROOT}/train", transform=tfm_train)
ds_val = MammogramFolders(f"{DATA_ROOT}/val", transform=tfm_eval)
ds_test = MammogramFolders(f"{DATA_ROOT}/test", transform=tfm_eval)

dl_train = DataLoader(
    ds_train, batch_size=16, shuffle=True, num_workers=2, pin_memory=True
)
dl_val = DataLoader(
    ds_val, batch_size=32, shuffle=False, num_workers=2, pin_memory=True
)
dl_test = DataLoader(
    ds_test, batch_size=32, shuffle=False, num_workers=2, pin_memory=True
)

# model/loss/opt (⚠️ use_attn=False)
model = build_model(pretrained=True, use_attn=False).to(DEVICE)
pos_weight = torch.tensor([ds_train.pos_weight()], device=DEVICE)
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)
scaler = GradScaler(device=DEVICE if DEVICE == "cuda" else "cpu")


def eval_metrics(model, loader):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(DEVICE, non_blocking=True)
            y = y.to(DEVICE)
            p = torch.sigmoid(model(x))
            ys.append(y.float().cpu().numpy())
            ps.append(p.float().cpu().numpy())
    y_true = np.concatenate(ys)
    y_prob = np.concatenate(ps)
    y_pred = (y_prob >= 0.5).astype(np.float32)
    auc = roc_auc_score(y_true, y_prob)
    f1 = f1_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sen = tp / (tp + fn + 1e-8)
    spe = tn / (tn + fp + 1e-8)
    return {"AUC": auc, "F1": f1, "ACC": acc, "SEN": sen, "SPE": spe}


EPOCHS = 5
best_auc = -1
os.makedirs("/content/runs", exist_ok=True)
ckpt_noattn = "/content/runs/baseline_noattn.pt"

for ep in range(1, EPOCHS + 1):
    model.train()
    t0 = time.time()
    run = 0
    n = 0
    for x, y in dl_train:
        x = x.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE)
        optimizer.zero_grad(set_to_none=True)
        with autocast(device_type="cuda", enabled=(DEVICE == "cuda")):
            logits = model(x)
            loss = criterion(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        run += loss.item() * x.size(0)
        n += x.size(0)
    scheduler.step()
    tr_loss = run / max(1, n)
    mv = eval_metrics(model, dl_val)
    print(
        f"[NoAttn Ep{ep}/{EPOCHS}] loss={tr_loss:.4f} | "
        f"val: AUC={mv['AUC']:.3f}, F1={mv['F1']:.3f}, ACC={mv['ACC']:.3f}, "
        f"SEN={mv['SEN']:.3f}, SPE={mv['SPE']:.3f} | {time.time()-t0:.1f}s"
    )
    if mv["AUC"] > best_auc:
        best_auc = mv["AUC"]
        torch.save({"model": model.state_dict(), "metrics_val": mv}, ckpt_noattn)
        print("  ✅ Saved best baseline:", ckpt_noattn)

# TEST metrics for baseline
state = torch.load(ckpt_noattn, map_location=DEVICE, weights_only=False)
model.load_state_dict(state["model"])
model.eval()
mb = eval_metrics(model, dl_test)
print(
    f"\nBASELINE (NoAttn, 5ep) TEST -> AUC={mb['AUC']:.3f} | F1={mb['F1']:.3f} | "
    f"ACC={mb['ACC']:.3f} | SEN={mb['SEN']:.3f} | SPE={mb['SPE']:.3f}"
)
print("Checkpoint:", ckpt_noattn)

[NoAttn Ep1/5] loss=0.4838 | val: AUC=0.932, F1=0.846, ACC=0.853, SEN=0.807, SPE=0.897 | 158.9s
  ✅ Saved best baseline: /content/runs/baseline_noattn.pt
[NoAttn Ep2/5] loss=0.3286 | val: AUC=0.962, F1=0.903, ACC=0.897, SEN=0.950, SPE=0.845 | 159.3s
  ✅ Saved best baseline: /content/runs/baseline_noattn.pt
[NoAttn Ep3/5] loss=0.2396 | val: AUC=0.969, F1=0.914, ACC=0.911, SEN=0.942, SPE=0.880 | 156.3s
  ✅ Saved best baseline: /content/runs/baseline_noattn.pt
[NoAttn Ep4/5] loss=0.2202 | val: AUC=0.975, F1=0.919, ACC=0.916, SEN=0.955, SPE=0.877 | 153.7s
  ✅ Saved best baseline: /content/runs/baseline_noattn.pt
[NoAttn Ep5/5] loss=0.1864 | val: AUC=0.976, F1=0.934, ACC=0.934, SEN=0.937, SPE=0.930 | 159.2s
  ✅ Saved best baseline: /content/runs/baseline_noattn.pt

BASELINE (NoAttn, 5ep) TEST -> AUC=0.970 | F1=0.929 | ACC=0.930 | SEN=0.922 | SPE=0.937
Checkpoint: /content/runs/baseline_noattn.pt


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# B5) Fair comparison (both 5 epochs) on TEST
import torch
from dataset import MammogramFolders
from model import build_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader
from torchvision import transforms

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATA_ROOT = "/content/data"
CKPT_ATTN = "/content/runs/attn_5ep.pt"
CKPT_NOAT = "/content/runs/baseline_noattn.pt"

tfm = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ]
)
dl_test = DataLoader(
    MammogramFolders(f"{DATA_ROOT}/test", transform=tfm),
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
)


def eval_ckpt(path, use_attn):
    m = build_model(pretrained=False, use_attn=use_attn).to(DEVICE)
    st = torch.load(path, map_location=DEVICE, weights_only=False)
    m.load_state_dict(st["model"])
    m.eval()
    Ys, Ps = [], []
    with torch.no_grad():
        for x, y in dl_test:
            x = x.to(DEVICE)
            y = y.to(DEVICE)
            p = torch.sigmoid(m(x))
            Ys.append(y.float().cpu().numpy())
            Ps.append(p.float().cpu().numpy())
    y_true = np.concatenate(Ys)
    y_prob = np.concatenate(Ps)
    y_pred = (y_prob >= 0.5).astype(np.float32)
    auc = roc_auc_score(y_true, y_prob)
    f1 = f1_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sen = tp / (tp + fn + 1e-8)
    spe = tn / (tn + fp + 1e-8)
    return {
        "AUC": float(auc),
        "F1": float(f1),
        "ACC": float(acc),
        "SEN": float(sen),
        "SPE": float(spe),
    }


m_attn = eval_ckpt(CKPT_ATTN, use_attn=True)
m_noatt = eval_ckpt(CKPT_NOAT, use_attn=False)

print("=== TEST (5 epochs each) ===")
print("+Attention :", m_attn)
print("NoAttention:", m_noatt)

# bar chart
metrics = ["AUC", "F1", "ACC", "SEN", "SPE"]
attn_vals = [m_attn[k] for k in metrics]
noatt_vals = [m_noatt[k] for k in metrics]

import numpy as np

plt.figure(figsize=(7, 3))
x = np.arange(len(metrics))
w = 0.35
plt.bar(x - w / 2, attn_vals, width=w, label="+Attention (5ep)")
plt.bar(x + w / 2, noatt_vals, width=w, label="Baseline (5ep)")
plt.xticks(x, metrics)
plt.ylim(0.80, 1.00)
for i, v in enumerate(attn_vals):
    plt.text(i - w / 2, v + 0.005, f"{v:.3f}", ha="center", fontsize=8)
for i, v in enumerate(noatt_vals):
    plt.text(i + w / 2, v + 0.005, f"{v:.3f}", ha="center", fontsize=8)
plt.ylabel("Score")
plt.title("Fair Comparison on TEST (5 epochs each)")
plt.legend()
plt.tight_layout()
fig_path = "/content/compare_equal.png"
plt.savefig(fig_path, dpi=180)
plt.close()
print("Saved figure:", fig_path)

=== TEST (5 epochs each) ===
+Attention : {'AUC': 0.9733249999999999, 'F1': 0.9247842170160296, 'ACC': 0.92375, 'SEN': 0.9374999999765625, 'SPE': 0.90999999997725}
NoAttention: {'AUC': 0.970375, 'F1': 0.929471032745592, 'ACC': 0.93, 'SEN': 0.9224999999769374, 'SPE': 0.9374999999765625}
Saved figure: /content/compare_equal.png


In [ ]:
# G1) Grad-CAM for +Attention (5ep) on a few TEST images
import os

import matplotlib.pyplot as plt
import numpy as np
import torch
from dataset import MammogramFolders
from model import build_model
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATA_ROOT = "/content/data"
CKPT = "/content/runs/attn_5ep.pt"
OUTDIR = "/content/gradcam_equal"
os.makedirs(OUTDIR, exist_ok=True)

# data
tfm = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ]
)
ds = MammogramFolders(f"{DATA_ROOT}/test", transform=tfm)

# model
model = build_model(pretrained=False, use_attn=True).to(DEVICE).eval()
state = torch.load(CKPT, map_location=DEVICE, weights_only=False)
model.load_state_dict(state["model"])

# last conv for Grad-CAM
target_module = model.backbone.layer4[-1].conv2
feats, grads = [], []
h1 = target_module.register_forward_hook(lambda m, i, o: feats.append(o.detach()))
h2 = target_module.register_full_backward_hook(
    lambda m, gin, gout: grads.append(gout[0].detach())
)


def gradcam(img_t):
    feats.clear()
    grads.clear()
    img_t = img_t.unsqueeze(0).to(DEVICE)
    model.zero_grad(set_to_none=True)
    logits = model(img_t)
    prob = torch.sigmoid(logits)[0]
    prob.backward(retain_graph=True)
    A = feats[-1][0]  # [C,H,W]
    G = grads[-1][0]  # [C,H,W]
    w = G.mean(dim=(1, 2), keepdim=True)
    cam = (A * w).sum(dim=0).cpu().numpy()
    cam = np.maximum(cam, 0)
    cam = cam / (cam.max() + 1e-8)
    return float(prob.detach().cpu()), cam


def read_raw(path):
    return np.array(Image.open(path).convert("L").resize((512, 512)))


# مسیرها طبق ترتیب داخلی دیتاست
paths = [p for p, _ in ds.samples]

# انتخاب: 2×TP, 2×TN, 1×FP, 1×FN (با اطمینان بالا)
samples = {"TP": [], "TN": [], "FP": [], "FN": []}
with torch.no_grad():
    for idx in range(len(ds)):
        x, y = ds[idx]
        p = torch.sigmoid(model(x.unsqueeze(0).to(DEVICE)))[0].item()
        pred = 1.0 if p >= 0.5 else 0.0
        key = (
            "TP"
            if (pred == 1 and y.item() == 1)
            else "TN"
            if (pred == 0 and y.item() == 0)
            else "FP"
            if (pred == 1 and y.item() == 0)
            else "FN"
        )
        samples[key].append((idx, float(p)))

samples["TP"] = sorted(samples["TP"], key=lambda t: -t[1])[:2]
samples["TN"] = sorted(samples["TN"], key=lambda t: t[1])[:2]
samples["FP"] = sorted(samples["FP"], key=lambda t: -t[1])[:1]
samples["FN"] = sorted(samples["FN"], key=lambda t: t[1])[:1]

count = 0
for k in ["TP", "TN", "FP", "FN"]:
    for idx, _ in samples[k]:
        x, y = ds[idx]
        prob, cam = gradcam(x)
        raw = read_raw(paths[idx])
        plt.figure(figsize=(4, 4))
        plt.imshow(raw, cmap="gray")
        plt.imshow(cam, alpha=0.35)
        title = f"{k} | prob={prob:.2f} | true={int(y.item())} | pred={int(prob>=0.5)}"
        plt.title(title)
        plt.axis("off")
        fname = os.path.join(OUTDIR, f"{k.lower()}_{idx:05d}.png")
        plt.savefig(fname, dpi=160, bbox_inches="tight")
        plt.close()
        print("Saved:", fname, "::", title)
        count += 1

h1.remove()
h2.remove()
print(f"\nTotal saved: {count} | Folder: {OUTDIR}")

Saved: /content/gradcam_equal/tp_00573.png :: TP | prob=1.00 | true=1 | pred=1
Saved: /content/gradcam_equal/tp_00684.png :: TP | prob=1.00 | true=1 | pred=1
Saved: /content/gradcam_equal/tn_00252.png :: TN | prob=0.00 | true=0 | pred=0
Saved: /content/gradcam_equal/tn_00369.png :: TN | prob=0.00 | true=0 | pred=0
Saved: /content/gradcam_equal/fp_00358.png :: FP | prob=0.98 | true=0 | pred=1
Saved: /content/gradcam_equal/fn_00538.png :: FN | prob=0.00 | true=1 | pred=0

Total saved: 6 | Folder: /content/gradcam_equal


In [ ]:
# A1) Train +Attention up to 15 epochs with Early-Stopping (patience=3)
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
from dataset import MammogramFolders
from model import build_model
from sklearn.metrics import roc_auc_score
from torch.amp import GradScaler
from torch.amp import autocast
from torch.utils.data import DataLoader
from torchvision import transforms

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATA_ROOT = "/content/data"
EPOCHS = 15
PATIENCE = 3

tfm_train = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomRotation(5),
        transforms.ToTensor(),
    ]
)
tfm_eval = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ]
)

ds_train = MammogramFolders(f"{DATA_ROOT}/train", transform=tfm_train)
ds_val = MammogramFolders(f"{DATA_ROOT}/val", transform=tfm_eval)
dl_train = DataLoader(
    ds_train, batch_size=16, shuffle=True, num_workers=2, pin_memory=True
)
dl_val = DataLoader(
    ds_val, batch_size=32, shuffle=False, num_workers=2, pin_memory=True
)

model = build_model(pretrained=True, use_attn=True).to(DEVICE)
pos_weight = torch.tensor([ds_train.pos_weight()], device=DEVICE)
crit = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
opt = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=EPOCHS)
scaler = GradScaler(device=DEVICE if DEVICE == "cuda" else "cpu")


def val_auc(m, loader):
    m.eval()
    Ys, Ps = [], []
    with torch.no_grad():
        for x, y in loader:
            p = torch.sigmoid(m(x.to(DEVICE)))
            Ys.append(y.float().numpy())
            Ps.append(p.float().cpu().numpy())
    y = np.concatenate(Ys)
    p = np.concatenate(Ps)
    return float(roc_auc_score(y, p))


hist = {"loss": [], "auc": []}
best_auc, best_ep, patience = -1, -1, 0
os.makedirs("/content/runs", exist_ok=True)
CKPT = "/content/runs/attn_early.pt"

for ep in range(1, EPOCHS + 1):
    t0 = time.time()
    model.train()
    run = 0
    n = 0
    for x, y in dl_train:
        x = x.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE)
        opt.zero_grad(set_to_none=True)
        with autocast(device_type="cuda", enabled=(DEVICE == "cuda")):
            loss = crit(model(x), y)
        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()
        run += loss.item() * x.size(0)
        n += x.size(0)
    sch.step()
    L = run / max(1, n)
    A = val_auc(model, dl_val)
    hist["loss"].append(L)
    hist["auc"].append(A)
    print(f"[Ep {ep}/{EPOCHS}] loss={L:.4f} | val AUC={A:.3f} | {time.time()-t0:.1f}s")
    if A > best_auc:
        best_auc, best_ep, patience = A, ep, 0
        torch.save({"model": model.state_dict(), "best_ep": ep, "val_auc": A}, CKPT)
        print("  ✅ Saved best:", CKPT)
    else:
        patience += 1
        if patience >= PATIENCE:
            print(f"⏹ Early-stopped at epoch {ep} (best={best_ep})")
            break

# curves
plt.figure(figsize=(6, 3))
ax1 = plt.gca()
ax2 = ax1.twinx()
ax1.plot(hist["loss"], label="Train Loss")
ax2.plot(hist["auc"], label="Val AUC", color="tab:orange")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax2.set_ylabel("AUC")
ln1, lb1 = ax1.get_legend_handles_labels()
ln2, lb2 = ax2.get_legend_handles_labels()
plt.legend(ln1 + ln2, lb1 + lb2, loc="center right")
plt.title("Curves (+Attention, EarlyStopping)")
plt.tight_layout()
plt.savefig("/content/curves_attn15.png", dpi=180)
plt.close()
print("Saved curves: /content/curves_attn15.png")
print(f"Best val AUC={best_auc:.3f} at epoch {best_ep} | Checkpoint={CKPT}")

[Ep 1/15] loss=0.5192 | val AUC=0.926 | 162.3s
  ✅ Saved best: /content/runs/attn_early.pt
[Ep 2/15] loss=0.3281 | val AUC=0.960 | 156.9s
  ✅ Saved best: /content/runs/attn_early.pt
[Ep 3/15] loss=0.2743 | val AUC=0.954 | 153.7s
[Ep 4/15] loss=0.2410 | val AUC=0.975 | 158.9s
  ✅ Saved best: /content/runs/attn_early.pt
[Ep 5/15] loss=0.2053 | val AUC=0.975 | 155.5s
[Ep 6/15] loss=0.1958 | val AUC=0.976 | 154.0s
  ✅ Saved best: /content/runs/attn_early.pt
[Ep 7/15] loss=0.1875 | val AUC=0.973 | 155.1s
[Ep 8/15] loss=0.1480 | val AUC=0.977 | 159.5s
  ✅ Saved best: /content/runs/attn_early.pt
[Ep 9/15] loss=0.1400 | val AUC=0.978 | 155.3s
  ✅ Saved best: /content/runs/attn_early.pt
[Ep 10/15] loss=0.1276 | val AUC=0.982 | 155.5s
  ✅ Saved best: /content/runs/attn_early.pt
[Ep 11/15] loss=0.1091 | val AUC=0.983 | 152.6s
  ✅ Saved best: /content/runs/attn_early.pt
[Ep 12/15] loss=0.0953 | val AUC=0.984 | 156.6s
  ✅ Saved best: /content/runs/attn_early.pt
[Ep 13/15] loss=0.0925 | val AUC=0.98

In [ ]:
# A2) Pick operating thresholds from VAL, then evaluate on TEST
import os

import matplotlib.pyplot as plt
import numpy as np
import torch
from dataset import MammogramFolders
from model import build_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from torch.utils.data import DataLoader
from torchvision import transforms

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATA_ROOT = "/content/data"
CKPT = "/content/runs/attn_early.pt"

# --- data ---
tfm = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ]
)
ds_val = MammogramFolders(f"{DATA_ROOT}/val", transform=tfm)
ds_test = MammogramFolders(f"{DATA_ROOT}/test", transform=tfm)
dl_val = DataLoader(
    ds_val, batch_size=64, shuffle=False, num_workers=2, pin_memory=True
)
dl_test = DataLoader(
    ds_test, batch_size=64, shuffle=False, num_workers=2, pin_memory=True
)

# --- model ---
state = torch.load(CKPT, map_location=DEVICE, weights_only=False)
model = build_model(pretrained=False, use_attn=True).to(DEVICE).eval()
model.load_state_dict(state["model"])


def collect_probs(loader):
    Ys, Ps = [], []
    with torch.no_grad():
        for x, y in loader:
            p = torch.sigmoid(model(x.to(DEVICE))).float().cpu().numpy()
            Ys.append(y.float().cpu().numpy())
            Ps.append(p)
    y = np.concatenate(Ys)
    p = np.concatenate(Ps)
    return y, p


y_val, p_val = collect_probs(dl_val)
y_test, p_test = collect_probs(dl_test)

# --- thresholds from VAL ---
fpr, tpr, thr = roc_curve(y_val, p_val)
auc_val = roc_auc_score(y_val, p_val)

# 1) default
t_default = 0.5
# 2) Youden's J
j = tpr - fpr
t_j = float(thr[np.argmax(j)])
# 3) target SEN ≈ 0.95  (closest)
target_sen = 0.95
idx_sen = np.argmin(np.abs(tpr - target_sen))
t_sen95 = float(thr[idx_sen])


def metrics_at_threshold(y_true, y_prob, t):
    y_pred = (y_prob >= t).astype(np.float32)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sen = tp / (tp + fn + 1e-8)
    spe = tn / (tn + fp + 1e-8)
    acc = (tp + tn) / (tp + tn + fp + fn + 1e-8)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_prob)
    return dict(
        TH=t,
        AUC=float(auc),
        F1=float(f1),
        ACC=float(acc),
        SEN=float(sen),
        SPE=float(spe),
        TP=int(tp),
        TN=int(tn),
        FP=int(fp),
        FN=int(fn),
    )


print(f"VAL AUC={auc_val:.3f}")
Ts = [("default_0.50", t_default), ("youdenJ", t_j), ("sen≈0.95", t_sen95)]

# --- evaluate on TEST for each threshold ---
results = {}
for name, tt in Ts:
    results[name] = metrics_at_threshold(y_test, p_test, tt)
    print(name, "->", results[name])

# --- plot ROC with operating points (from VAL) ---
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr, label=f"Val ROC (AUC={auc_val:.3f})")
plt.plot([0, 1], [0, 1], "--", alpha=0.5)


# mark points
def point_for_t(t):
    # find nearest threshold index for plotting
    i = np.argmin(np.abs(thr - t))
    return fpr[i], tpr[i]


for name, tt in Ts:
    x, y = point_for_t(tt)
    plt.scatter([x], [y], label=f"{name} (t={tt:.2f})")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("VAL ROC with chosen thresholds")
plt.legend(loc="lower right")
plt.tight_layout()
out_fig = "/content/roc_op_points.png"
plt.savefig(out_fig, dpi=180)
plt.close()
print("Saved figure:", out_fig)

VAL AUC=0.985
default_0.50 -> {'TH': 0.5, 'AUC': 0.98006875, 'F1': 0.9432098765432099, 'ACC': 0.9424999999882188, 'SEN': 0.954999999976125, 'SPE': 0.92999999997675, 'TP': 382, 'TN': 372, 'FP': 28, 'FN': 18}
youdenJ -> {'TH': 0.6269708871841431, 'AUC': 0.98006875, 'F1': 0.9389788293897883, 'ACC': 0.9387499999882656, 'SEN': 0.9424999999764375, 'SPE': 0.934999999976625, 'TP': 377, 'TN': 374, 'FP': 26, 'FN': 23}
sen≈0.95 -> {'TH': 0.7321080565452576, 'AUC': 0.98006875, 'F1': 0.9423558897243107, 'ACC': 0.9424999999882188, 'SEN': 0.9399999999765, 'SPE': 0.944999999976375, 'TP': 376, 'TN': 378, 'FP': 22, 'FN': 24}
Saved figure: /content/roc_op_points.png


In [ ]:
import os

import matplotlib.pyplot as plt

# A3) Pretty table + Confusion Matrices for the three thresholds
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

# نتایج مرحله قبل را اینجا کپی می‌کنیم (عیناً از لاگ شما)
res = {
    "default_0.50": {
        "TH": 0.5,
        "AUC": 0.98006875,
        "F1": 0.9432098765432099,
        "ACC": 0.9424999999882188,
        "SEN": 0.954999999976125,
        "SPE": 0.92999999997675,
        "TP": 382,
        "TN": 372,
        "FP": 28,
        "FN": 18,
    },
    "youdenJ": {
        "TH": 0.6269708871841431,
        "AUC": 0.98006875,
        "F1": 0.9389788293897883,
        "ACC": 0.9387499999882656,
        "SEN": 0.9424999999764375,
        "SPE": 0.934999999976625,
        "TP": 377,
        "TN": 374,
        "FP": 26,
        "FN": 23,
    },
    "sen≈0.95": {
        "TH": 0.7321080565452576,
        "AUC": 0.98006875,
        "F1": 0.9423558897243107,
        "ACC": 0.9424999999882188,
        "SEN": 0.9399999999765,
        "SPE": 0.944999999976375,
        "TP": 376,
        "TN": 378,
        "FP": 22,
        "FN": 24,
    },
}

# 1) جدول تمیز
df = pd.DataFrame(res).T[
    ["TH", "AUC", "F1", "ACC", "SEN", "SPE", "TP", "TN", "FP", "FN"]
]
df = df.rename(
    index={
        "default_0.50": "default(0.50)",
        "youdenJ": "youdenJ(~0.63)",
        "sen≈0.95": "sen≈0.95(~0.73)",
    }
)
print(df.round(3))
df.to_csv("/content/threshold_comparison.csv", index=True)
print("Saved CSV -> /content/threshold_comparison.csv")


# 2) CM figure helper
def save_cm(tp, tn, fp, fn, title, path):
    mat = np.array([[tn, fp], [fn, tp]])
    plt.figure(figsize=(4, 4))
    plt.imshow(mat, cmap="viridis")
    for i in range(2):
        for j in range(2):
            plt.text(
                j, i, str(mat[i, j]), ha="center", va="center", color="w", fontsize=14
            )
    plt.xticks([0, 1], ["Pred 0", "Pred 1"])
    plt.yticks([0, 1], ["True 0", "True 1"])
    plt.title(title)
    plt.tight_layout()
    plt.savefig(path, dpi=180)
    plt.close()
    print("Saved:", path)


os.makedirs("/content", exist_ok=True)
save_cm(
    **{
        "tp": res["default_0.50"]["TP"],
        "tn": res["default_0.50"]["TN"],
        "fp": res["default_0.50"]["FP"],
        "fn": res["default_0.50"]["FN"],
    },
    title="Confusion Matrix – default (t=0.50)",
    path="/content/cm_default.png",
)

save_cm(
    **{
        "tp": res["youdenJ"]["TP"],
        "tn": res["youdenJ"]["TN"],
        "fp": res["youdenJ"]["FP"],
        "fn": res["youdenJ"]["FN"],
    },
    title="Confusion Matrix – YoudenJ (~0.63)",
    path="/content/cm_youden.png",
)

save_cm(
    **{
        "tp": res["sen≈0.95"]["TP"],
        "tn": res["sen≈0.95"]["TN"],
        "fp": res["sen≈0.95"]["FP"],
        "fn": res["sen≈0.95"]["FN"],
    },
    title="Confusion Matrix – SEN≈0.95 (~0.73)",
    path="/content/cm_sen95.png",
)

                    TH   AUC     F1    ACC    SEN    SPE     TP     TN    FP  \
default(0.50)    0.500  0.98  0.943  0.942  0.955  0.930  382.0  372.0  28.0   
youdenJ(~0.63)   0.627  0.98  0.939  0.939  0.942  0.935  377.0  374.0  26.0   
sen≈0.95(~0.73)  0.732  0.98  0.942  0.942  0.940  0.945  376.0  378.0  22.0   

                   FN  
default(0.50)    18.0  
youdenJ(~0.63)   23.0  
sen≈0.95(~0.73)  24.0  
Saved CSV -> /content/threshold_comparison.csv
Saved: /content/cm_default.png
Saved: /content/cm_youden.png
Saved: /content/cm_sen95.png
